# 成績データファイルを作成する
## 元の成績データファイルへ新しい成績データを追加する

In [ ]:
import os
import pandas as pd
from tkinter import Tk, filedialog

# 使用するファイルパスの指定
basedir = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Original'
filename = 'ResultData_Master_2023.csv'

master_filepath = os.path.join(basedir, filename)

# ファイル選択ダイアログを表示するための準備
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする
root.attributes('-topmost', True) # ダイアログを最前面に出す（隠れるのを防ぐため）

merged_filepath = filedialog.askopenfilename(title="CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])
print(merged_filepath)

# ファイルが選択されなかった場合は終了
if not merged_filepath:
    print("ファイルが選択されなかったため、処理を終了します。")
    exit()

# CSVファイルを開く
master_df = pd.read_csv(master_filepath, encoding='cp932').copy()
merged_df = pd.read_csv(merged_filepath, encoding='cp932').copy()

# 開いたファイルの列をリネームする
# リネームする列名を定義
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid'
}

master_df.rename(columns=rename_map, inplace=True)
merged_df.rename(columns=rename_map, inplace=True)

# master_dfにmerged_dfを縦結合する
combined_df = pd.concat([master_df, merged_df], ignore_index=True)

# 重複行を削除する（全列が同じ行を削除）
combined_df.drop_duplicates(inplace=True)

# master_dfを上書き保存する
# ※保存は元のマスターに合わせて cp932 にしています
combined_df.to_csv(master_filepath, index=False, encoding='cp932')

print("処理が完了しました。")

G:/マイドライブ/20_HOBBY/20_KEIBA/10_Data_Source/10_Export_Data/Result_Data/Original/ResultData_2023.csv


C:\Users\rotte\AppData\Local\Temp\ipykernel_8332\272507429.py:26: DtypeWarning: Columns (24,25,101) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(master_filepath, encoding='cp932').copy()


処理が完了しました。


In [2]:
import os
import pandas as pd
# 使用するファイルパスの指定
basedir = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Original'
filename = 'ResultData_Master_2023.csv'

master_filepath = os.path.join(basedir, filename)

master_df = pd.read_csv(master_filepath, encoding='cp932')

# 開いたファイルの列をリネームする
# リネームする列名を定義
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid'
}

master_df.rename(columns=rename_map, inplace=True)

# csvファイル保存
master_df.to_csv(master_filepath, index=False, encoding='cp932')

C:\Users\rotte\AppData\Local\Temp\ipykernel_24020\4066021823.py:9: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  master_df = pd.read_csv(master_filepath, encoding='cp932')


# 追切指数の作成

## 追切指数基準タイムを作成する

In [10]:
import pandas as pd
import numpy as np
import os

# =========================
# 設定
# =========================
output_dir = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\\'
traning_filepath = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\50_Training_Comments\Training_Comments_Master.csv'
master_filepath = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Original\ResultData_Master_2023.csv'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# =========================
# 読み込み
# =========================
df1 = pd.read_csv(traning_filepath, encoding='cp932').copy()
df2 = pd.read_csv(master_filepath, encoding='cp932').copy()

# =========================
# 前処理
# =========================
def clean_training_data(df):
    df = df.copy()
    mask_invalid = df["日付"].astype(str).str.contains("■|◇", na=False)
    df = df[~mask_invalid].copy()
    
    check_cols = [c for c in ['8F','7F','6F','5F(4F)','4F(3F)','3F(2F)'] if c in df.columns]
    for c in check_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    
    is_not_hill = ~df['コース'].astype(str).str.contains('坂', na=False)
    has_10sec_like = np.column_stack([df[c].between(1.0, 19.9) for c in check_cols]).any(axis=1)
    
    drop_mask = is_not_hill & has_10sec_like
    df = df[~drop_mask].copy()
    return df

df1 = clean_training_data(df1)

replace_map = {'南Ｗ': '美Ｗ', '南Ｄ': '美ダ', '南ダ': '美ダ', '南芝': '美芝'}
df1['コース'] = df1['コース'].replace(replace_map)
df1['馬場状態'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
df1["回り位置"] = pd.to_numeric(df1["回り位置"], errors="coerce")

distance_columns = [col for col in ["8F", "7F", "6F", "5F(4F)", "4F(3F)", "3F(2F)", "1F"] if col in df1.columns]

for col in distance_columns:
    df1[f"{col}_補"] = np.nan

for index, row in df1.iterrows():
    if not pd.isna(row["回り位置"]):
        correction_value = (9 - row["回り位置"]) * 0.1
        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)
            df1.at[index, f"{leftmost_col}_補"] = round(row[leftmost_col] + correction_value, 1)
            
            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        df1.at[index, f"{col}_補"] = round(row[col] + equal_correction, 1)
        else:
            continue
    else:
        for col in distance_columns:
            if not pd.isna(row[col]):
                df1.at[index, f"{col}_補"] = row[col]

# =========================
# マージ
# =========================
merge_cols = ['target_horseid', 'トラックコード(JV)', '年齢限定(競走種別コード)', 'クラスコード', '入線順位']
df2 = df2[merge_cols]

def categorize_race_type(race_type):
    if race_type == 11: return 'サラブレッド系2歳'
    elif race_type == 12: return 'サラブレッド系3歳'
    elif race_type >= 13: return 'サラブレッド系3歳以上'
    else: return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15: return '新馬・未勝利'
    elif class_code == 23: return '1勝クラス'
    elif class_code == 43: return '2勝クラス'
    elif class_code == 67: return '3勝クラス'
    elif class_code >= 114: return 'OP・重賞'
    else: return np.nan

df2['年齢限定(競走種別コード)'] = df2['年齢限定(競走種別コード)'].apply(categorize_race_type)
df2['クラスコード'] = df2['クラスコード'].apply(categorize_class_code)

merged_df1 = pd.merge(df1, df2, on="target_horseid", how="inner")
del df1, df2

merged_df1 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
time_columns = ['8F_補', '7F_補', '6F_補', '5F(4F)_補', '4F(3F)_補', '3F(2F)_補', '1F_補']

# =========================
# 基準タイム・MAD計算
# =========================

# MAD（中央値絶対偏差）計算用関数
def robust_mad(x):
    return np.median(np.abs(x - np.median(x)))

# 並び替え用の列リストを定義
mad_columns = [f"MAD_{c}" for c in time_columns]

# 1. コース・馬場状態ごとの集計 (Median + MAD)
course_stats = merged_df1.groupby(['コース', '馬場状態'])[time_columns].agg(['median', robust_mad])

new_cols_course = []
for col, stat in course_stats.columns:
    if stat == 'median':
        new_cols_course.append(col)
    else:
        new_cols_course.append(f"MAD_{col}") 
course_stats.columns = new_cols_course
course_stats = course_stats.reset_index().round(2) # 精度確保のため2桁

# 列の並び替え
course_stats = course_stats[['コース', '馬場状態'] + time_columns + mad_columns]

# 2. コース・クラス・馬場状態ごとの集計 (Median + MAD)
class_stats = merged_df1.groupby(['コース', 'クラスコード', '馬場状態'])[time_columns].agg(['median', robust_mad])

new_cols_class = []
for col, stat in class_stats.columns:
    if stat == 'median':
        new_cols_class.append(col)
    else:
        new_cols_class.append(f"MAD_{col}")
class_stats.columns = new_cols_class
class_stats = class_stats.reset_index().round(2)

# 列の並び替え
class_stats = class_stats[['コース', 'クラスコード', '馬場状態'] + time_columns + mad_columns]

# 3. 上位20% / 10% の平均
def calculate_top20times_mean(group):
    top20_threshold = group[time_columns].quantile(0.2)
    top20_data = group[time_columns][group[time_columns] <= top20_threshold]
    return top20_data.mean()

def calculate_top10times_mean(group):
    top10_threshold = group[time_columns].quantile(0.1)
    top10_data = group[time_columns][group[time_columns] <= top10_threshold]
    return top10_data.mean()

course_top20_mean = round(merged_df1.groupby(['コース', '馬場状態']).apply(calculate_top20times_mean), 1).reset_index()
course_top10_mean = round(merged_df1.groupby(['コース', '馬場状態']).apply(calculate_top10times_mean), 1).reset_index()

# =========================
# 出力
# =========================
course_stats.to_csv(f"{output_dir}traning_std_course.csv", index=False, encoding="cp932")
class_stats.to_csv(f"{output_dir}traning_std_class.csv", index=False, encoding="cp932")
course_top20_mean.to_csv(f"{output_dir}traning_std_course_top20.csv", index=False, encoding="cp932")
course_top10_mean.to_csv(f"{output_dir}traning_std_course_top10.csv", index=False, encoding="cp932")

del merged_df1, course_stats, class_stats, course_top20_mean, course_top10_mean

print("完了：基準タイムファイル（MAD版）を出力しました。")

C:\Users\rotte\AppData\Local\Temp\ipykernel_24020\2642273033.py:19: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(master_filepath, encoding='cp932').copy()
C:\Users\rotte\AppData\Local\Temp\ipykernel_24020\2642273033.py:155: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  course_top20_mean = round(merged_df1.groupby(['コース', '馬場状態']).apply(calculate_top20times_mean), 1).reset_index()
C:\Users\rotte\AppData\Local\Temp\ipykernel_24020\2642273033.py:156: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from t

完了：基準タイムファイル（MAD版）を出力しました。


## 追切指数を算出する

In [ ]:
import pandas as pd
import numpy as np

# =========================
# 読み込みファイルパス
# =========================
traning_filepath = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\50_Training_Comments\Training_Comments_Master.csv'
master_filepath  = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Original\ResultData_Master_2023.csv'

# 基準ファイル
p_course_median  = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_course.csv'
p_class_median   = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_class.csv'
p_course_top20   = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_course_top20.csv'
p_course_top10   = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_course_top10.csv'

# 出力
output_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\210_Training_Score\Training_Score_Master.csv'
output_dir       = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\210_Training_Score\\'

# =========================
# 読み込み & 前処理
# =========================
df1 = pd.read_csv(traning_filepath, encoding='cp932').copy()
df2 = pd.read_csv(master_filepath,  encoding='cp932').copy()

def clean_training_data(df):
    df = df.copy()
    mask_invalid = df["日付"].astype(str).str.contains("■|◇", na=False)
    df = df[~mask_invalid].copy()
    check_cols = [c for c in ['8F','7F','6F','5F(4F)','4F(3F)','3F(2F)'] if c in df.columns]
    for c in check_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    is_not_hill = ~df['コース'].astype(str).str.contains('坂', na=False)
    has_10sec_like = np.column_stack([df[c].between(10.0, 19.9) for c in check_cols]).any(axis=1)
    drop_mask = is_not_hill & has_10sec_like
    df = df[~drop_mask].copy()
    return df

df1 = clean_training_data(df1)

replace_map = {'南Ｗ': '美Ｗ', '南Ｄ': '美ダ', '南ダ': '美ダ', '南芝': '美芝'}
df1['コース'] = df1['コース'].replace(replace_map)
df1['馬場状態'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
df1["回り位置"] = pd.to_numeric(df1["回り位置"], errors="coerce")

distance_columns = [col for col in ["8F", "7F", "6F", "5F(4F)", "4F(3F)", "3F(2F)", "1F"] if col in df1.columns]
for col in distance_columns:
    df1[f"{col}_補"] = np.nan

for index, row in df1.iterrows():
    if not pd.isna(row["回り位置"]):
        correction_value = (9 - row["回り位置"]) * 0.1
        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)
            df1.at[index, f"{leftmost_col}_補"] = round(row[leftmost_col] + correction_value, 1)
            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        df1.at[index, f"{col}_補"] = round(row[col] + equal_correction, 1)
        else:
            continue
    else:
        for col in distance_columns:
            if not pd.isna(row[col]):
                df1.at[index, f"{col}_補"] = row[col]

time_columns = [f"{c}_補" for c in distance_columns]

# =========================
# 成績マージ
# =========================
merge_cols = ['target_horseid', 'トラックコード(JV)', '年齢限定(競走種別コード)', 'クラスコード']
df2 = df2[merge_cols]

def categorize_race_type(race_type):
    if race_type == 11: return 'サラブレッド系2歳'
    elif race_type == 12: return 'サラブレッド系3歳'
    elif race_type >= 13: return 'サラブレッド系3歳以上'
    else: return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15: return '新馬・未勝利'
    elif class_code == 23: return '1勝クラス'
    elif class_code == 43: return '2勝クラス'
    elif class_code == 67: return '3勝クラス'
    elif class_code >= 114: return 'OP・重賞'
    else: return np.nan

df2['年齢限定(競走種別コード)'] = df2['年齢限定(競走種別コード)'].apply(categorize_race_type)
df2['クラスコード'] = df2['クラスコード'].apply(categorize_class_code)

merged_df1 = pd.merge(df1, df2, on='target_horseid', how='inner').copy()
merged_df1['is_saka'] = merged_df1['コース'].astype(str).str.contains('坂', na=False)

# =========================
# 基準テーブル読み込み（MAD対応）
# =========================
def load_and_tag(path, suffix, has_std):
    t = pd.read_csv(path, encoding='cp932').copy()
    rename_map = {}
    for c in time_columns:
        if c in t.columns:
            rename_map[c] = f"{c}_{suffix}"
        
        # 標準偏差ではなくMADを探す
        madc = f"MAD_{c}"
        if has_std and (madc in t.columns):
            rename_map[madc] = f"MAD_{c}_{suffix}"
    return t.rename(columns=rename_map)

df3 = load_and_tag(p_course_median, 'course', True)
df4 = load_and_tag(p_class_median,  'class',  True)
df5 = load_and_tag(p_course_top20, 'course_20', False)
df6 = load_and_tag(p_course_top10, 'course_10', False)

key_course = ['コース','馬場状態']
key_class  = ['年齢限定(競走種別コード)','クラスコード','コース','馬場状態']

def smerge(left, right, keys):
    exist_keys = [k for k in keys if (k in left.columns and k in right.columns)]
    return pd.merge(left, right, on=exist_keys, how='left')

merged_df1 = smerge(merged_df1, df3, key_course)
merged_df1 = smerge(merged_df1, df4, key_class)
merged_df1 = smerge(merged_df1, df5, key_course)
merged_df1 = smerge(merged_df1, df6, key_course)

# =========================
# 偏差値（ロバスト偏差値計算：MAD使用）
# =========================
def dev_vec(v, m, mad):
    # MADを正規分布の標準偏差相当に変換する定数 1.4826
    sigma_est = mad * 1.4826
    
    ok = (~pd.isna(v)) & (~pd.isna(m)) & (~pd.isna(mad)) & (sigma_est != 0)
    
    # タイムは小さい方が良いので (Median - Value)
    # sigma_est が 0 (全員同じタイム等) の場合は偏差値50とする
    return np.where(ok, 50 + 10 * (m - v) / sigma_est, 50)

bases_for_dev = ['course', 'class']

for c in time_columns:
    dev_cols = []
    for b in bases_for_dev:
        mean_col = f"{c}_{b}"
        mad_col  = f"MAD_{c}_{b}"
        colname = f"偏差値_{c}_{b}"
        
        # MADが存在する場合のみ計算
        if mad_col in merged_df1.columns:
            merged_df1[colname] = dev_vec(merged_df1[c], merged_df1[mean_col], merged_df1[mad_col])
            dev_cols.append(colname)

    if dev_cols:
        merged_df1[f"偏差値_統合_{c}"] = merged_df1[dev_cols].mean(axis=1, skipna=True)

# 統合偏差値が存在する列のみでスコア計算
valid_dev_cols = [f"偏差値_統合_{c}" for c in time_columns if f"偏差値_統合_{c}" in merged_df1.columns]
if valid_dev_cols:
    merged_df1['総合偏差値スコア'] = merged_df1[valid_dev_cols].mean(axis=1, skipna=True)
else:
    merged_df1['総合偏差値スコア'] = 50.0

# =========================
# 加点（変更なし）
# =========================
bonus_targets = {'4F': '4F(3F)_補', '2F': '3F(2F)_補', '1F': '1F_補'}

def calculate_bonus(value, mean, is_saka, col_short):
    if pd.isna(value) or pd.isna(mean): return 0.0
    if is_saka:
        if col_short == '4F': return 0.5 if value < mean else 0.0
        if col_short == '2F': return 1.5 if value < mean else 0.0
        if col_short == '1F': return 1.0 if value < mean else 0.0
    else:
        if col_short == '4F': return 0.5 if value < mean else 0.0
        if col_short == '2F': return 1.0 if value < mean else 0.0
        if col_short == '1F': return 1.5 if value < mean else 0.0
    return 0.0

th_sfx_list = ['course_20','course_10']
bonus_cols = []
for col_short, base_col in bonus_targets.items():
    for sfx in th_sfx_list:
        th_col = f"{base_col}_{sfx}"
        if th_col in merged_df1.columns:
            out_col = f"加点_{col_short}_{sfx}"
            merged_df1[out_col] = merged_df1.apply(
                lambda r: calculate_bonus(
                    r.get(base_col, np.nan), r.get(th_col, np.nan),
                    bool(r.get('is_saka', False)), col_short
                ), axis=1
            )
            bonus_cols.append(out_col)

merged_df1['総合加点スコア'] = merged_df1[bonus_cols].sum(axis=1, skipna=True) if bonus_cols else 0.0

# =========================
# 係数・最終指数
# =========================
def rider_coef(x):
    s = '' if pd.isna(x) else str(x)
    if '助手' in s: return 1.0
    if '見習' in s: return 0.8
    return 0.9

def leg_coef(x):
    s = '' if pd.isna(x) else str(x)
    if '馬なり' in s: return 1.1
    if ('Ｇ' in s) or ('G' in s) or ('強' in s): return 1.0
    if '一杯' in s: return 0.8
    if 'ヨレ' in s: return 0.7
    if 'バテ' in s: return 0.6
    return 0.9

merged_df1['騎乗者係数'] = merged_df1['乗り役'].apply(rider_coef)
merged_df1['脚色係数']   = merged_df1['脚色'].apply(leg_coef)

merged_df1['追切指数'] = (
    (merged_df1['総合偏差値スコア'].fillna(50) + merged_df1['総合加点スコア'].fillna(0.0))
    * merged_df1['騎乗者係数'].fillna(0.9)
    * merged_df1['脚色係数'].fillna(0.9)
)

merged_df1['追切指数'] = merged_df1['追切指数'].replace([np.inf, -np.inf], np.nan).fillna(1).round(1)

# 重複排除
merged_unique = merged_df1.loc[merged_df1.groupby('target_horseid')['追切指数'].idxmax()].reset_index(drop=True).copy()
merged_unique = merged_unique[['target_horseid','追切指数']].copy()

del df1,df2,df3,df4,df5,df6,merged_df1

# 保存処理
combined_df = pd.read_csv(output_filepath1, encoding='cp932').copy()
combined_df = pd.concat([combined_df, merged_unique], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset='target_horseid', keep='last')
combined_df.to_csv(output_filepath1, index=False, encoding='cp932')

combined_df['year'] = pd.to_datetime(
    combined_df['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y', errors='coerce'
).dt.year

for year, dfy in combined_df.groupby('year'):
    dfy.drop(columns='year').to_csv(f'{output_dir}Training_Score_Master_{year}.csv', index=False, encoding='cp932')

del combined_df

print("完了：追切指数（MAD・ロバスト偏差値版）を出力しました。")

C:\Users\rotte\AppData\Local\Temp\ipykernel_24020\1744245144.py:24: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(master_filepath,  encoding='cp932').copy()
C:\Users\rotte\AppData\Local\Temp\ipykernel_24020\1744245144.py:240: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([combined_df, merged_unique], ignore_index=True)


完了：追切指数（MAD・ロバスト偏差値版）を出力しました。


# 基準タイムファイルを出力するコード

## ・スピード指数の算出に使うコース別の基準タイムファイルを出力する
- 1着馬および1着馬〜3着馬の基準タイム、およびその他タイム系データを格納する
- ファイルは2つ出力するが、スピード指数の算出には`00_StandardTimes2_yyyymmdd.csv`を用いる

### 処理の概要
- 成績CSVファイルを読み込み、コース・距離・競争種別コード・クラスコード・馬場状態でレースデータをグループ化する
- 各グループの中央値を基準タイムファイル1とする
- 各グループの中で1勝クラス・2勝クラスのデータの平均をとり、それを基準タイムファイル2とする

### ステップ
1. **CSVファイルの読み込み**
   - `pandas`を使って、ある期間の1着馬から3着馬の成績が記録されたCSVファイルを読み込む

2. **カテゴリの追加**
   - `競争種別コード`と`クラスコード`をもとにレースタイプやクラスカテゴリを新たに分類し、カテゴリカラムを追加
   - 馬場状態（良・稍／重・不良）に基づき馬場状態カテゴリを追加

3. **タイム系カラムの計算**
   - 1着馬から3着馬の走破タイムや3Fの平均を計算して新しいカラムとして追加

4. **グループ化して中央値を計算**
   - 「場所」、「芝・ダート」、「距離」、「馬場状態」、「競争種別カテゴリ」、「クラスカテゴリ」でグループ化
   - 各グループごとのタイム系カラムの中央値を計算し、基準タイムファイル1として保存

5. **クラス別フィルタリングと再計算**
   - 基準タイムファイル1から「良・稍」と「重・不良」でデータをフィルタリング
   - 各フィルタで1勝クラスと2勝クラスの中央値を取得し、その平均をとって基準タイムファイル2とする

6. **出力ファイルの生成**
   - `00_StandardTimes1_yyyymmdd.csv`（基準タイムファイル1）と`00_StandardTimes2_yyyymmdd.csv`（基準タイムファイル2）を出力
   - スピード指数の算出には`00_StandardTimes2_yyyymmdd.csv`を使用


In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import re
import os

#---ステップ１：成績データから基準タイムファイルを作成
# ファイルパスの指定
# 成績データファイルのパス
master_file_path = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Original\ResultData_Master_2023.csv'

# 成績データにマージするデータファイルのパス
# 初角位置ファイルのパス
merge_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\70_First_Corner_Position\First_Corner_Position_Master.csv'
# 2角位置ファイルのパス
merge_filepath2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\80_Second_Corner_Position\Second_Corner_Position_Master.csv'
# 3角位置ファイルのパス
merge_filepath3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\90_Third_Corner_Position\Third_Corner_Position_Master.csv'
# 4角位置ファイルのパス
merge_filepath4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\100_Fourth_Corner_Position\Fourth_Corner_Position_Master.csv'
# 上り位置ファイルのパス
merge_filepath5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\110_Spurt_Position\Spurt_Position_Master.csv'
# 馬場指数ファイルのパス
merge_filepath6 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\60_Track_Condition\Track_Condition_Master.csv'
# 前半3Fタイムファイルのパス
merge_filepath7 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\120_First3F_Lap\First3F_Lap_Master.csv'
# 追切指数ファイルのパス
merge_filepath8 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\210_Training_Score\Training_Score_Master.csv'

# 保存先パス
# 保存先のファイルパス
# 加工後の成績データファイルの保存先パス
output_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Processed\Race_Result_Master_2024_tmp.csv'

# 基準ファイル系
# 基準タイムファイル1の保存先パス
output_filepath2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\StdTime1.csv'
# 基準タイムファイル2の保存先パス
output_filepath3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\StdTime2.csv'
# ペース係数ファイルの保存先パス
output_filepath4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\PaceTime.csv'
# 基準33ラップファイルの保存先パス
output_filepath5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\33Lap.csv'
# レースレベル基準ファイルのパス
output_filepath6 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\RaceLevel.csv'

# 馬タイプ分類のファイルパス
output_filepath7 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\220_Horse_Type\Horse_Type_Master.csv'

# 指数系
# テン指数のディレクトリ
output_dir1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\150_First_Score\\'
# 上り指数のディレクトリ
output_dir2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\160_Spurt_Score\\'
# スピード指数のディレクトリ
output_dir3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\170_Speed_Score\\'
# 総合指数のディレクトリ
output_dir4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\180_Total_Score\\'
# 33ラップ判定のディレクトリ
output_dir5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\140_33Lap_Category\\'
# レースレベル判定のディレクトリ
output_dir6 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\190_Race_Level\\'

# 列の並び定義ファイルパス
header_filepath = r'G:\マイドライブ\20_HOBBY\20_KEIBA\50_SourceCode\ResultData_Header.csv'

# 成績データファイルとマージファイルの読み込み
df1 = pd.read_csv(master_file_path, encoding='cp932')
df2 = pd.read_csv(merge_filepath1, encoding='cp932')
df3 = pd.read_csv(merge_filepath2, encoding='cp932')
df4 = pd.read_csv(merge_filepath3, encoding='cp932')
df5 = pd.read_csv(merge_filepath4, encoding='cp932')
df6 = pd.read_csv(merge_filepath5, encoding='cp932')
df7 = pd.read_csv(merge_filepath6, encoding='cp932')
df8 = pd.read_csv(merge_filepath7, encoding='cp932')
df9 = pd.read_csv(merge_filepath8, encoding='cp932')

#　マージファイルで使用する列だけを定義
df2 = df2[['target_horseid','初角サイドポジション']]
df3 = df3[['target_horseid','2角サイドポジション']]
df4 = df4[['target_horseid','3角サイドポジション']]
df5 = df5[['target_horseid','4角サイドポジション']]
df6 = df6[['target_horseid','4角位置']]
df7 = df7[['target_raceid','馬場指数']]
df8 = df8[['target_horseid','前半3F']]
df9 = df9[['target_horseid','追切指数']]

# カラムの整理
# 成績データファイルの列名を変更
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid',
    '上り3F': 'レース上り3F',
    '上り4F': 'レース上り4F',
    '上り5F': 'レース上り5F',
    '外部指数1':'レイティング',
    '外部指数順1':'レイティング順位',
    '外部指数2':'ZI指数',
    '外部指数順2':'ZI指数順位',
    '外部指数3':'追切指数',
    '外部指数順3':'追切指数順位',
    '上り3F.1':'上り3F'
}

# df1の列名に対してrename_mapを適用
df1 = df1.rename(columns=rename_map)

# キーの型を揃える（数字/ゼロ埋めブレ防止）
for c in ['target_horseid', 'target_raceid']:
    if c in df1: df1[c] = df1[c].astype(str)
for d in [df2, df3, df4, df5, df6, df7, df8, df9]:
    for c in ['target_horseid', 'target_raceid']:
        if c in d: d[c] = d[c].astype(str)

#　種牡馬名のspace削除
df1['種牡馬'] = df1['種牡馬'].astype(str).str.replace(r'\s+', '', regex=True)

# ラベル化する列を追加
# コースラベル
insert_pos = df1.columns.get_loc('コースグループ名1') + 1
df1.insert(insert_pos, 'コースラベル', df1['場所'].astype(str) + '_' + df1['芝・ダート'].astype(str) + '_' + df1['距離'].astype(str) + '_' + df1['トラックコード(JV)'].astype(str))

# 父×母の父タイプ名
insert_pos = df1.columns.get_loc('母の父タイプ名') + 1
df1.insert(insert_pos, '父×母の父タイプ名', df1['種牡馬'].astype(str) + '×' + df1['母の父タイプ名'].astype(str))

# 父タイプ名×母の父タイプ名
insert_pos = df1.columns.get_loc('父×母の父タイプ名') + 1
df1.insert(insert_pos, '父タイプ名×母の父タイプ名', df1['種牡馬タイプ名'].astype(str) + '×' + df1['母の父タイプ名'].astype(str))

# 生産者×馬主
insert_pos = df1.columns.get_loc('騎手') + 1
df1.insert(insert_pos, '生産者×馬主', df1['生産者'].astype(str) + '×' + df1['馬主'].astype(str))

# 生産者×調教師
insert_pos = df1.columns.get_loc('生産者×馬主') + 1
df1.insert(insert_pos, '生産者×調教師', df1['生産者'].astype(str) + '×' + df1['調教師'].astype(str))

# 生産者×騎手
insert_pos = df1.columns.get_loc('生産者×調教師') + 1
df1.insert(insert_pos, '生産者×騎手', df1['生産者'].astype(str) + '×' + df1['騎手'].astype(str))

# 馬主×調教師
insert_pos = df1.columns.get_loc('生産者×騎手') + 1
df1.insert(insert_pos, '馬主×調教師', df1['馬主'].astype(str) + '×' + df1['調教師'].astype(str))

# 馬主×騎手
insert_pos = df1.columns.get_loc('馬主×調教師') + 1
df1.insert(insert_pos, '馬主×騎手', df1['馬主'].astype(str) + '×' + df1['騎手'].astype(str))

# 調教師×騎手
insert_pos = df1.columns.get_loc('馬主×騎手') + 1
df1.insert(insert_pos, '調教師×騎手', df1['調教師'].astype(str) + '×' + df1['騎手'].astype(str))

# 調教師×コースラベル
insert_pos = df1.columns.get_loc('調教師×騎手') + 1
df1.insert(insert_pos, '調教師×コースラベル', df1['調教師'].astype(str) + '×' + df1['コースラベル'].astype(str))

# 騎手×コースラベル
insert_pos = df1.columns.get_loc('調教師×コースラベル') + 1
df1.insert(insert_pos, '騎手×コースラベル', df1['騎手'].astype(str) + '×' + df1['コースラベル'].astype(str))

# 配当金額の列を作成する
# 列から取り出す金額の数を定義
target_cols = {
    '単勝配当表記': ('単勝', 1),
    '複勝配当表記': ('複勝', 3),
    '枠連配当表記': ('枠連', 1),
    '馬連配当表記': ('馬連', 1),
    'ワイド配当表記': ('ワイド', 3),
    '馬単配当表記': ('馬単', 1),
    '３連複配当表記': ('３連複', 1),
    '３連単配当表記': ('３連単', 1),
}

# 金額を取り出す正規表現パターン
pattern = re.compile(r'[\\¥]\s*([0-9,]+)(?=\s*(?:\(|/|$))')

def pick_amounts(text, take=1):
    if pd.isna(text):
        return [np.nan]*take
    s = str(text)
    found = pattern.findall(s)
    nums = []
    for x in found:
        try:
            nums.append(int(x.replace(',', '')))
        except:
            # 変な値が来てもスルー
            continue
    # 必要な個数だけ先頭から取り、足りなければNaNで埋める
    nums = nums[:take]
    if len(nums) < take:
        nums += [np.nan]*(take - len(nums))
    return nums

for src_col, (base_name, take) in target_cols.items():
    if src_col not in df1.columns:
        # その列が無い場合はスキップ
        continue

    # 値を取り出す
    values = df1[src_col].apply(lambda x: pick_amounts(x, take))

    # 1個だけなら「単勝」のように1列、3個なら「複勝1, 複勝2, 複勝3」の3列を作る
    if take == 1:
        col_name = base_name
        df1[col_name] = values.apply(lambda v: v[0] if isinstance(v, list) else np.nan)
        # Convert to integer, coercing errors to NaN
        df1[col_name] = pd.to_numeric(df1[col_name], errors='coerce').astype('Int64')
    else:
        for i in range(take):
            col_name = f'{base_name}{i+1}'
            df1[col_name] = values.apply(lambda v: v[i] if isinstance(v, list) and len(v) > i else np.nan)
            # Convert to integer, coercing errors to NaN
            df1[col_name] = pd.to_numeric(df1[col_name], errors='coerce').astype('Int64')

# 成績データファイルへ対象データをマージする
# 初角位置のマージ
df1 = pd.merge(df1, df2, on='target_horseid', how='left')
# 2角位置のマージ
df1 = pd.merge(df1, df3, on='target_horseid', how='left')
# 3角位置のマージ
df1 = pd.merge(df1, df4, on='target_horseid', how='left')
# 4角位置のマージ
df1 = pd.merge(df1, df5, on='target_horseid', how='left')
# 上り位置のマージ
df1 = pd.merge(df1, df6, on='target_horseid', how='left')
# 馬場指数のマージ
df1 = pd.merge(df1, df7, on='target_raceid', how='left')
# 前半3Fのマージ
df1 = pd.merge(df1, df8, on='target_horseid', how='left')
# 追切指数の代入 とりあえずTrueで上書き
left  = df1.set_index('target_horseid')
right = df9[['target_horseid','追切指数']].dropna(subset=['追切指数']).set_index('target_horseid')
left.update(right, overwrite=True)
df1 = left.reset_index()

# --- 関数群 ---
def calculate_33_lap(df1):
    """距離ごとに33ラップを計算する"""
    rap_33 = []
    distance_to_lap_range = {
        1000: (0, 3),
        1200: (0, 3),
        1300: (1, 4),
        1400: (1, 4),
        1500: (2, 5),
        1600: (2, 5),
        1700: (3, 6),
        1800: (3, 6),
        1900: (4, 7),
        2000: (4, 7),
        2100: (5, 8),
        2200: (5, 8),
        2300: (6, 9),
        2400: (6, 9),
        2500: (7, 10),
        2600: (7, 10),
        2700: (8, 11),
        2800: (8, 11),
        2900: (9, 12),
        3000: (9, 12),
        3100: (10, 13),
        3200: (10, 13),
        3300: (11, 14),
        3400: (11, 14),
        3500: (12, 15),
        3600: (12, 15),
    }
    for _, row in df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_lap_range:
            start, end = distance_to_lap_range[distance]
            sum_6to4 = sum(lap_times[start:end])
        elif distance == 1150:
            sum_6to4 = round(lap_times[0] * 1.25, 1) + lap_times[1] + lap_times[2] if len(lap_times) >= 3 else None
        else:
            sum_6to4 = None
        sum_3to1 = row['レース上り3F'] if 'レース上り3F' in row and not pd.isna(row['レース上り3F']) else None
        rap_33_value = sum_6to4 - sum_3to1 if not pd.isna(sum_6to4) and not pd.isna(sum_3to1) else None
        rap_33.append(rap_33_value)
    df1['33ラップ'] = rap_33
    return df1

def calculate_middle_lap(df1):
    """距離ごとに中盤ラップ1・2を計算する"""
    middle_lap1 = []
    middle_lap2 = []
    distance_to_mid_lap = {
        1000: (2, 4, None, None),
        1150: (2, 4, None, None),
        1200: (2, 4, None, None),
        1300: (2, 4, None, None),
        1400: (2, 4, None, None),
        1500: (3, 5, None, None),
        1600: (3, 5, None, None),
        1700: (3, 6, None, None),
        1800: (3, 6, None, None),
        1900: (3, 7, None, None),
        2000: (3, 7, None, None),
        2100: (3, 5, 5, 8),
        2200: (3, 5, 5, 8),
        2300: (3, 5, 5, 8),
        2400: (3, 6, 6, 9),
        2500: (3, 6, 6, 9),
        2600: (3, 7, 7, 10),
        3000: (3, 8, 8, 12),
        3200: (3, 8, 8, 13),
        3400: (3, 9, 9, 14),
        3600: (3, 9, 9, 15),
    }
    for _, row in df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_mid_lap:
            mid1_start, mid1_end, mid2_start, mid2_end = distance_to_mid_lap[distance]
            mid1 = sum(lap_times[mid1_start:mid1_end]) if mid1_start is not None else None
            mid2 = sum(lap_times[mid2_start:mid2_end]) if mid2_start is not None else None
        else:
            mid1 = None
            mid2 = None
        middle_lap1.append(mid1)
        middle_lap2.append(mid2)
    df1['中盤ラップ1'] = middle_lap1
    df1['中盤ラップ2'] = middle_lap2
    return df1

def calculate_lap_features(df):
    """
    Lap01～Lap25 を使って
      ・最大加速（隣接ラップ差分の最小値）
      ・ゴール前ラップ差（ラスト1F - ラスト2F）
    を計算して df に列を追加する。
    """
    lap_cols = [f'Lap{str(i).zfill(2)}' for i in range(1, 26)]

    def _calc_row(row):
        # その馬のラップ一覧（NaN は除外）
        laps = []
        for c in lap_cols:
            if c in row.index and pd.notna(row[c]):
                laps.append(row[c])

        # ラップが1つ以下ならどっちも計算不能
        if len(laps) < 2:
            return pd.Series({'最大加速ラップ': np.nan, 'ゴール前ラップ差': np.nan})

        laps = np.array(laps, dtype=float)

        # 隣り合う差分（後ろ - 前）
        diffs = np.diff(laps)   # 例：Lap02-Lap01, Lap03-Lap02, ...

        # 最大加速 = 最もマイナスが大きい差分（＝最小値）
        max_accel = diffs.min() if len(diffs) > 0 else np.nan

        # 終盤ラップ差 = ラスト1F - ラスト2F
        last_diff = laps[-1] - laps[-2] if len(laps) >= 2 else np.nan

        return pd.Series({
            '最大加速ラップ': max_accel if pd.notna(max_accel) else np.nan,
            'ゴール前ラップ差': last_diff if pd.notna(last_diff) else np.nan
        })

    new_cols = df.apply(_calc_row, axis=1)
    df['最大加速ラップ'] = new_cols['最大加速ラップ'].round(1)
    df['ゴール前ラップ差'] = new_cols['ゴール前ラップ差'].round(1)

    return df

def calculate_days_since_birth(df1):
    """生後日数を計算する"""
    birth_days = []
    for _, row in df1.iterrows():
        race_date = datetime.strptime(row['日付S'], '%Y.%m.%d')
        birth_str = row['誕生日'].replace(" ", "").replace("日", "").replace("-", "")
        birth_month, birth_day = map(int, birth_str.replace("月", " ").split())
        birth_year = race_date.year - row['年齢']
        try:
            birth_date = datetime(birth_year, birth_month, birth_day)
        except ValueError:
            birth_date = datetime(birth_year, 2, 28)
        days_old = (race_date - birth_date).days
        birth_days.append(days_old)
    df1['生後日数'] = birth_days
    return df1

def calculate_distance_diff(df1):
    """前走距離との差を計算する"""
    df1['前走距離差'] = df1['距離'] - df1['前走距離']
    return df1

def calculate_firsthalf_diff(df1):
    """初角から4角位置の差分を計算"""
    df1['初角_4角差'] = df1.apply(lambda row:
        (row['通過1'] - row['通過4']) if pd.notna(row['通過1']) else \
        ((row['通過2'] - row['通過4']) if pd.notna(row['通過2']) else \
        ((row['通過3'] - row['通過4']) if pd.notna(row['通過3']) else np.nan))
    , axis=1).fillna(0)
    return df1

def calculate_goal_diff(df1):
    """4角から入線順位の差分を計算"""
    # 入線順位を一時的に数値化
    rank_num = pd.to_numeric(df1['入線順位'], errors='coerce')

    # 有効な順位（1以上）だけを判定するためのマスク
    valid_mask = rank_num >= 1

    # 出力列だけ作る
    df1['4角_入線順位差'] = np.nan

    # 有効な行だけ計算
    df1.loc[valid_mask, '4角_入線順位差'] = (
        pd.to_numeric(df1.loc[valid_mask, '通過4'], errors='coerce')
        - rank_num[valid_mask]
    )

    return df1

def calculate_sideposition(df1):
    """サイドポジションの平均を計算する"""
    cols = ['初角サイドポジション', '2角サイドポジション', '3角サイドポジション', '4角サイドポジション']
    df1['サイドポジション平均'] = df1[cols].mean(axis=1)
    return df1

def calculate_totalprize(df1):
    """獲得賞金を計算する"""
    df1['獲得賞金'] = df1['賞金'].fillna(0) + df1['付加賞金'].fillna(0)
    return df1

def convert_time_to_seconds(time_str):
    """タイム表記を秒数に変換"""
    try:
        parts = time_str.split(".")
        if len(parts) == 3:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
        else:
            return np.nan
        return total_seconds
    except:
        return np.nan

def remove_plus_sign(value):
    """数値データから `+` を削除して変換"""
    try:
        return float(str(value).replace("+", ""))
    except:
        return np.nan

def extract_weight(value):
    """斤量の数値部分だけを抽出"""
    try:
        match = re.search(r'\d+', str(value))
        return int(match.group()) if match else np.nan
    except:
        return np.nan

def calculate_corner_loss(row):
    """コーナーロスを計算"""
    corner_positions = [
        row.get('初角サイドポジション', 1) - 1,
        row.get('2角サイドポジション', 1) - 1,
        row.get('3角サイドポジション', 1) - 1,
        row.get('4角サイドポジション', 1) - 1
    ]
    total_distance_loss = sum(corner_positions) * 1.5  # m単位
    finish_time_seconds = row['タイムS']
    distance_m = row['距離']
    if pd.isna(finish_time_seconds) or pd.isna(distance_m) or finish_time_seconds == 0:
        return np.nan
    avg_speed = distance_m / finish_time_seconds if finish_time_seconds > 0 else np.nan
    corner_loss = total_distance_loss / avg_speed if avg_speed > 0 else 0
    return round(corner_loss, 2)

# 各列の整形
# 対象の列を数値変換
df1['馬場指数'] = df1['馬場指数'].astype(str).str.extract(r'(-?\d+)')[0].astype('Int64')
df1['レイティング'] = pd.to_numeric(df1['レイティング'], errors='coerce')
df1['体重'] = pd.to_numeric(df1['体重'], errors='coerce')
df1['Ave-3F'] = pd.to_numeric(df1['Ave-3F'], errors='coerce')
df1['上り3F'] = pd.to_numeric(df1['上り3F'], errors='coerce')

# 対象の列から記号を除去
for col in ['前後3F差', '前後4F差', '前後5F差', '増減']:
    df1[col] = df1[col].apply(remove_plus_sign)

df1['斤量'] = df1['斤量'].apply(extract_weight)

# タイムを秒数に変換
df1['タイムS'] = df1['タイムS'].apply(convert_time_to_seconds)
df1['-3Fタイム'] = df1['-3Fタイム'].apply(convert_time_to_seconds)

# 決め手列を変換(マップにない場合はNanにする)
lq_mapping = {
    '中団': '差し',
    '後方': '追込',
}

s = df1['決め手'].astype('string').str.strip()
df1['決め手'] = s.map(lq_mapping)

# 各種計算関数を順次実行
# 33ラップの計算
df1 = calculate_33_lap(df1)
# 中盤ラップ1・2の計算
df1 = calculate_middle_lap(df1)
# 最大加速ラップ・ゴール前ラップ差の計算
df1 = calculate_lap_features(df1)
# 生後日数の計算
df1 = calculate_days_since_birth(df1)
# 前走距離差の計算
df1 = calculate_distance_diff(df1)
# 初角から4角通過順位差の計算
df1 = calculate_firsthalf_diff(df1)
# 4角から入線順位差の計算
df1 = calculate_goal_diff(df1)
# サイドポジション平均値の計算
df1 = calculate_sideposition(df1)
# 獲得賞金の計算
df1 = calculate_totalprize(df1)
# 基準斤量の計算
df1['基準斤量'] = df1['斤量'] - df1['馬齢斤量差']
# RPCI差の計算
df1['RPCI差'] = df1['PCI'] - df1['レースPCI']
# コーナーロスの計算
df1['コーナーロス'] = df1.apply(calculate_corner_loss, axis=1)
# 補正走破タイムの計算
df1['補正走破タイム'] = df1['タイムS'] - df1['コーナーロス']
# スローorハイ関数の計算
df1['スローorハイ関数'] = df1['Ave-3F'] - df1['上り3F']

# マージ用の列作成処理
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 115:
        return 'OP・重賞'
    else:
        return np.nan

# 範囲定義
ranges = [ (-np.inf, -4.6), (-4.6, -3.6), (-3.6, -2.6), (-2.6, -1.6), (-1.6, -0.6), (-0.6, 0.6), (0.6, 1.6), (1.6, 2.6), (2.6, 3.6), (3.6, 4.6), (4.6, np.inf) ]

# 区間ラベル
def assign_range(value):
    if pd.isna(value):
        return np.nan
    for i, (lower, upper) in enumerate(ranges):
        if i < len(ranges) - 1:
            if lower <= value < upper:
                return f"{lower}～{upper}"
        else:
            # 最終区間（4.6～inf）は右も含める
            if lower <= value <= upper:
                return f"{lower}～{upper}"
    return np.nan  # 念のため

# 馬場分類・競走種別・クラス分類・スローor関数範囲の列を作成する
df1['馬場分類'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
df1['競走種別'] = df1['年齢限定(競走種別コード)'].apply(categorize_race_type)
df1['クラス分類'] = df1['クラスコード'].apply(categorize_class_code)
df1['スローorハイ関数範囲'] = df1['スローorハイ関数'].apply(assign_range)

# --- 集計設定 ---
agg_cols1 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    '追切指数': 'median',
    '追切指数順位': 'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols2 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング順位': 'median',
    '追切指数': 'median',
    '追切指数順位': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'マイニング': 'median',
    'マイニング順位': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング順位': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '上り3F': 'median',
    '上り3F順位': 'median',
    'Ave-3F': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理1 ---

# 1勝クラス・2勝クラスのみ対象
base_df1 = df1[df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# グループ化キー
group_cols1 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類', 'クラス分類']

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols1).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols2).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---

df_tops2 = df1[df1['入線順位'].between(1, 3)].copy()
df_1st2 = df1[df1['入線順位'] == 1].copy()

# グループ化キー
group_cols2 = ['場所', '芝・ダート', '距離','トラックコード(JV)', '競走種別', 'クラス分類', '馬場分類']

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols1).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols2).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols2.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# --- 距離係数（速度換算） ---
base_time_df2['距離係数'] = (1 / base_time_df2['タイムS']) * 100

# --- ペース係数算出処理 ---

# 外れ値除外（IQR）
def remove_outliers(group):
    Q1 = group['タイムS'].quantile(0.25)
    Q3 = group['タイムS'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return group[(group['タイムS'] >= lower_bound) & (group['タイムS'] <= upper_bound)]

# グルーピング＆IQR除外
pace_df = (
    df1.groupby(['場所', '芝・ダート', '距離','トラックコード(JV)'])
       .apply(remove_outliers)
       .reset_index(drop=True)
       .copy()
)

# 区間別の中央値（タイムS & スローorハイ関数）
pace_medians_df = pace_df.groupby(
    ['場所', '芝・ダート', '距離','トラックコード(JV)', 'スローorハイ関数範囲']
).agg({
    'タイムS': 'median',
    'スローorハイ関数': 'median'
}).reset_index()

# --- ベースライン作成 ---
# 優先：-0.6～0.6（中庸帯）
# 代用：0.6～1.6 / -1.6～-0.6 の平均 → どちらか片方だけでも可
baseline_medians = {}
for (location, turf_dirt, distance,track_code), group_df in pace_medians_df.groupby(['場所', '芝・ダート', '距離','トラックコード(JV)']):
    baseline_row = group_df[group_df['スローorハイ関数範囲'] == '-0.6～0.6']
    if not baseline_row.empty:
        baseline_median = float(baseline_row['タイムS'].iloc[0])
    else:
        range_negative = group_df[group_df['スローorハイ関数範囲'] == '-1.6～-0.6']
        range_positive = group_df[group_df['スローorハイ関数範囲'] == '0.6～1.6']
        if not range_negative.empty and not range_positive.empty:
            baseline_median = (float(range_negative['タイムS'].median()) +
                               float(range_positive['タイムS'].median())) / 2.0
        elif not range_negative.empty:
            baseline_median = float(range_negative['タイムS'].median())
        elif not range_positive.empty:
            baseline_median = float(range_positive['タイムS'].median())
        else:
            baseline_median = np.nan
    baseline_medians[(location, turf_dirt, distance,track_code)] = baseline_median

# ベースラインとの差（スローorハイ関数差指数）
def calculate_difference(row):
    key = (row['場所'], row['芝・ダート'], row['距離'],row['トラックコード(JV)'])
    baseline_time = baseline_medians.get(key, np.nan)
    return row['タイムS'] - baseline_time if not pd.isna(baseline_time) else np.nan

pace_medians_df['スローorハイ関数差指数'] = pace_medians_df.apply(calculate_difference, axis=1)

# 係数計算（0除算・NaN安全＋クリップ）
def calculate_pace_adjustment_coefficient(row):
    val = row['スローorハイ関数']
    diff = row['スローorハイ関数差指数']
    if pd.notna(val) and val != 0 and pd.notna(diff):
        coefficient = round(diff / val, 2)
        return float(np.clip(coefficient, -3, 3))
    return 0.0

pace_medians_df['ペース補正係数'] = pace_medians_df.apply(calculate_pace_adjustment_coefficient, axis=1)

# --- 基準33ラップ算出処理 ---

lap33_df = (
    df1
      .dropna(subset=['33ラップ'])  # まず欠損を除外
      .groupby(['場所', '芝・ダート', '距離', 'トラックコード(JV)'], dropna=False)['33ラップ']
      .agg(['mean', 'std'])  # mean と std を同時に集計
      .reset_index()
).copy()

# 近似Zスコア用の基準値を計算
lap33_df['33ラップ±0'] = lap33_df['mean'].round(2)
lap33_df['33ラップ+1'] = (lap33_df['mean'] + lap33_df['std']).round(2)
lap33_df['33ラップ-1'] = (lap33_df['mean'] - lap33_df['std']).round(2)
lap33_df['33ラップ+2'] = (lap33_df['mean'] + 2 * lap33_df['std']).round(2)
lap33_df['33ラップ-2'] = (lap33_df['mean'] - 2 * lap33_df['std']).round(2)

# 必要なカラムだけ残す
lap33_df = lap33_df[
    ['場所', '芝・ダート', '距離', 'トラックコード(JV)',
     '33ラップ-2', '33ラップ-1', '33ラップ±0', '33ラップ+1', '33ラップ+2',
     'mean', 'std']
]

#---ステップ２：成績データへ指数の追加
# 共通関数：参照側DFのキー以外の列にサフィックスを付ける
def prepare_ref_df(df_ref, merge_keys, suffix):
    """
    マージ用の参照DFの列名に suffix を付ける（キー列以外すべて）
    """
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    df_ref = df_ref.rename(columns=rename_map)
    return df_ref

# マージキー
merge_key1 = ['場所','芝・ダート','距離','トラックコード(JV)','馬場分類']
merge_key2 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類','馬場分類']
merge_key3 = ['場所','芝・ダート','距離','トラックコード(JV)','スローorハイ関数範囲']
merge_key4 = ['場所','芝・ダート','距離','トラックコード(JV)']

# 参照側の列名にsuffixを付与
ref_df1 = prepare_ref_df(base_time_df1, merge_key1, '_stdtime1').copy()
ref_df2 = prepare_ref_df(base_time_df2, merge_key2, '_stdtime2').copy()
ref_df3 = prepare_ref_df(pace_medians_df, merge_key3, '_paceindex').copy()
ref_df4 = prepare_ref_df(lap33_df, merge_key4, '_33lap').copy()

# df1へその他データフレームをマージ
merged_df1 = pd.merge(df1, ref_df1, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, ref_df2, on=merge_key2, how='left')
merged_df1 = pd.merge(merged_df1, ref_df3, on=merge_key3, how='left')
merged_df1 = pd.merge(merged_df1, ref_df4, on=merge_key4, how='left')

# 不要なデータフレームのクリア
del df1, ref_df1, ref_df2, ref_df3, ref_df4

# 各指数算出の前処理
# 基準タイム/距離係数のベース列を作る（Std1優先、なければStd2）
base_time = merged_df1['タイムS_stdtime1'].fillna(merged_df1['タイムS_stdtime2'])
dist_coef = merged_df1['距離係数_stdtime1'].fillna(merged_df1['距離係数_stdtime2'])

# 基準タイム差（秒）：基準タイム - 補正走破タイム
base_time_diff = base_time - merged_df1['補正走破タイム']

# 基準タイム差×距離係数
speed_core = base_time_diff * dist_coef

# 平均3F補正値
ave3f_base = merged_df1['Ave-3F_stdtime1'].fillna(merged_df1['Ave-3F_stdtime2'])
revi_Ave3F = ave3f_base - merged_df1['Ave-3F']

# 斤量補正値
revi_weight = merged_df1['斤量'] - merged_df1['基準斤量']

# クラス補正値（Std1が存在する行だけ有効。Std2代用行では0）
# ※ Std1が無い = その条件の1・2勝基準が作れない想定なので、クラス補正は入れない
revi_class = (merged_df1['タイムS_stdtime1'] - merged_df1['タイムS_stdtime2']).where(
    merged_df1['タイムS_stdtime1'].notna(), 0
)

# ペース補正値
revi_pace = merged_df1['スローorハイ関数'] * merged_df1['ペース補正係数_paceindex']

# 馬場指数秒数換算
merged_df1['馬場補正値'] = merged_df1['馬場指数'] / 10

# 初角位置
merged_df1['初角位置'] = (
    merged_df1['通過1']
      .fillna(merged_df1['通過2'])
      .fillna(merged_df1['通過3'])
      .fillna(merged_df1['通過4'])
)

# テン指数の計算
merged_df1['テン指数'] = round((
(merged_df1['前半3F_stdtime1'].fillna(merged_df1['前半3F_stdtime2']) - merged_df1['前半3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数'] - merged_df1['初角位置'] + 1) / merged_df1['頭数']) # 初角の位置取り評価
+ 50
), 1)

# 上り指数の計算
merged_df1['上り指数'] = round((
(merged_df1['上り3F_stdtime1'].fillna(merged_df1['上り3F_stdtime2']) - merged_df1['上り3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数']) - merged_df1['入線順位'] + 1) / merged_df1['頭数'] # 着順評価
+ (merged_df1['通過4'] - merged_df1['入線順位']) / merged_df1['頭数'] # ポジション押し上げ力評価
+ (merged_df1['頭数'] - merged_df1['通過4'] + 1) / merged_df1['頭数'] # 4角の位置取り評価
+ 50
), 1)

# スピード指数の計算 ---
merged_df1['スピード指数'] = (
    (
        speed_core
        + merged_df1['馬場補正値'].fillna(0)
        + revi_Ave3F.fillna(0)
        + revi_weight.fillna(0)
        + revi_class.fillna(0)
        + revi_pace.fillna(0)
        + 100
    )
    .round(1)
    .where(speed_core.notna() & (speed_core != 0))
)

# 総合指数の計算
merged_df1['総合指数'] = merged_df1[['スピード指数', '補正タイム', '補9']].mean(axis=1).round(1)

# 総合指数の代表値(1着～3着)をdf1へ追加
central_score_df1 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
central_score_df1 = central_score_df1.groupby('target_raceid' ,as_index = False)['総合指数'].mean()
central_score_df1.rename(columns={'総合指数': 'Top3総合指数'}, inplace=True)
central_score_df1['Top3総合指数'] = round(central_score_df1['Top3総合指数'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df1, on='target_raceid', how='left',suffixes=('', '_centralscore1')).copy()

# レイティングの平均値を成績データへ追加
merged_df1['レイティング'] = pd.to_numeric(merged_df1['レイティング'], errors='coerce')
central_score_df2 = merged_df1.groupby('target_raceid' ,as_index = False)['レイティング'].mean().copy()
central_score_df2.rename(columns={'レイティング': 'レイティング平均値'}, inplace=True)
central_score_df2['レイティング平均値'] = round(central_score_df2['レイティング平均値'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df2, on='target_raceid', how='left' ,suffixes=('', '_centralscore2')).copy()

# 不要なデータフレームのクリア
del central_score_df1, central_score_df2

# 各指数の偏差値を計算
def deviation_in_race(series):
    mean = series.mean()
    std = series.std()
    if pd.isna(std) or std == 0:
        return pd.Series(np.nan, index=series.index)
    return ((series - mean) / std * 10 + 50)

# レース内偏差値
merged_df1['レイティング偏差値'] = (merged_df1.groupby('target_raceid')['レイティング'].transform(deviation_in_race).round(1))
merged_df1['ZI指数偏差値'] = (merged_df1.groupby('target_raceid')['ZI指数'].transform(deviation_in_race).round(1))
merged_df1['追切指数偏差値'] = (merged_df1.groupby('target_raceid')['追切指数'].transform(deviation_in_race).round(1))
merged_df1['マイニング偏差値'] = (merged_df1.groupby('target_raceid')['マイニング'].transform(deviation_in_race).round(1))
merged_df1['対戦型マイニング偏差値'] = (merged_df1.groupby('target_raceid')['対戦型マイニング'].transform(deviation_in_race).round(1))
merged_df1['前半3F偏差値'] = merged_df1.groupby('target_raceid')['前半3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['上り3F偏差値'] = merged_df1.groupby('target_raceid')['上り3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['Ave-3F偏差値'] = merged_df1.groupby('target_raceid')['Ave-3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['テン指数偏差値'] = (merged_df1.groupby('target_raceid')['テン指数'].transform(deviation_in_race).round(1))
merged_df1['上り指数偏差値'] = (merged_df1.groupby('target_raceid')['上り指数'].transform(deviation_in_race).round(1))
merged_df1['スピード指数偏差値'] = (merged_df1.groupby('target_raceid')['スピード指数'].transform(deviation_in_race).round(1))
merged_df1['総合指数偏差値'] = (merged_df1.groupby('target_raceid')['総合指数'].transform(deviation_in_race).round(1))

# レース内順位（dense）
merged_df1['前半3F順位'] = (merged_df1.groupby('target_raceid')['前半3F'].rank(ascending=True, method='dense').astype('Int64'))
merged_df1['テン指数順位'] = (merged_df1.groupby('target_raceid')['テン指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['上り指数順位'] = (merged_df1.groupby('target_raceid')['上り指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['Ave-3F順位'] = (merged_df1.groupby('target_raceid')['Ave-3F'].rank(ascending=True, method='dense').astype('Int64'))
merged_df1['スピード指数順位'] = (merged_df1.groupby('target_raceid')['スピード指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['総合指数順位'] = (merged_df1.groupby('target_raceid')['総合指数'].rank(ascending=False, method='dense').astype('Int64'))

# 順位分布の計算
merged_df1['レイティング順位分布'] = (merged_df1['レイティング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['ZI指数順位分布'] = (merged_df1['ZI指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['追切指数順位分布'] = (merged_df1['追切指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['マイニング順位分布'] = (merged_df1['マイニング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['対戦型マイニング順位分布'] = (merged_df1['対戦型マイニング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['前半3F順位分布'] = (merged_df1['前半3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['テン指数順位分布'] = (merged_df1['テン指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['上り3F順位分布'] = (merged_df1['上り3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['上り指数順位分布'] = (merged_df1['上り指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['Ave-3F順位分布'] = (merged_df1['Ave-3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['スピード指数順位分布'] = (merged_df1['スピード指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['総合指数順位分布'] = (merged_df1['総合指数順位'] - 1) / (merged_df1['頭数'] - 1)

# 33ラップ判定用関数
def assign_label(row):
    lap_value = row['33ラップ']
    if pd.isna(lap_value):
        return np.nan

    candidates = {
        -2: row['33ラップ-2_33lap'],
        -1: row['33ラップ-1_33lap'],
         0: row['33ラップ±0_33lap'],
         1: row['33ラップ+1_33lap'],
         2: row['33ラップ+2_33lap'],
    }

    # 参照側が全部NaNなら判定不能
    if all(pd.isna(v) for v in candidates.values()):
        return np.nan

    # 差分（絶対値）が最小のスケールを選ぶ
    best_scale = min(
        candidates.keys(),
        key=lambda k: abs(lap_value - candidates[k]) if pd.notna(candidates[k]) else np.inf
    )

    # 0スケール
    if best_scale == 0:
        if lap_value < 0:
            return '持0'
        elif lap_value > 0:
            return '瞬0'
        else:
            return '総'

    prefix = '瞬' if lap_value > 0 else '持'
    suffix = f'+{best_scale}' if best_scale > 0 else f'{best_scale}'
    return prefix + suffix

# 33ラップを判定
merged_df1['33ラップ判定'] = merged_df1.apply(assign_label, axis=1)

# レースタイプラベル列の追加
insert_pos = merged_df1.columns.get_loc('33ラップ判定') + 1

# '33ラップ判定' の値 → レースタイプ の対応表
race_type_map = {
    # 瞬発力戦
    '瞬+2': '瞬発力戦',
    '瞬+1': '瞬発力戦',
    '瞬0':  '瞬発力戦',

    # 総合力戦
    '瞬-1': '総合力戦',
    '瞬-2': '総合力戦',
    '総':   '総合力戦',
    '持-1': '総合力戦',
    '持-2': '総合力戦',

    # 持久力戦
    '持+2': '持久力戦',
    '持+1': '持久力戦',
    '持+0': '持久力戦',
}

ref = merged_df1['33ラップ判定']
merged_df1.insert(insert_pos, 'レースタイプ', ref.map(race_type_map))

# ラベル書き換え用のマッピング辞書
label_mapping = {
    '持-2': '0T持-2',
    '持-1': '0T持-1',
    '持0': '0U持0',
    '持+1': '0V持+1',
    '持+2': '0V持+2',
    '瞬-2': '0S瞬-2',
    '瞬-1': '0S瞬-1',
    '瞬0': '0R瞬0',
    '瞬+1': '0Q瞬+1',
    '瞬+2': '0Q瞬+2',
    '総': '02総'
}

# 書き換え
merged_df1['レース印２'] = merged_df1['33ラップ判定'].replace(label_mapping)

# レース強度指数の計算
first_load  = merged_df1['通過3F_stdtime1'].fillna(merged_df1['通過3F_stdtime2']) - merged_df1['通過3F']

middle_diff1 = merged_df1['中盤ラップ1_stdtime1'].fillna(merged_df1['中盤ラップ1_stdtime2']) - merged_df1['中盤ラップ1']
middle_diff2 = merged_df1['中盤ラップ2_stdtime1'].fillna(merged_df1['中盤ラップ2_stdtime2']) - merged_df1['中盤ラップ2']

middle_load = pd.concat([middle_diff1, middle_diff2], axis=1).mean(axis=1, skipna=True)

last_load   = merged_df1['レース上り3F_stdtime1'].fillna(merged_df1['レース上り3F_stdtime2']) - merged_df1['レース上り3F']
spurt_load  = merged_df1['最大加速ラップ_stdtime1'].fillna(merged_df1['最大加速ラップ_stdtime2']) - merged_df1['最大加速ラップ']
goal_load   = merged_df1['ゴール前ラップ差_stdtime1'].fillna(merged_df1['ゴール前ラップ差_stdtime2']) - merged_df1['ゴール前ラップ差']

merged_df1['レース強度指数'] = (
    100
    + first_load.fillna(0)
    + middle_load.fillna(0)
    + last_load.fillna(0)
    + spurt_load.fillna(0)
    + goal_load.fillna(0)
    + merged_df1['馬場補正値'].fillna(0)
)

# サフィックス付き列を削除
# 削除するサフィックスを列挙
suffixes_to_drop = ('_1着','_stdtime1', '_stdtime2', '_paceindex', '_33lap', '_centralscore1', '_centralscore2')

# サフィックスで判定して残す列だけ抜き出し
merged_df1 = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

#---ステップ３：基準タイムファイル１・２の再集計
# --- 集計設定 ---
agg_cols3 = {
    'レースPCI': 'median',
    'PCI3': 'median',
    '通過3F': 'median',
    '通過4F': 'median',
    '通過5F': 'median',
    'レース上り3F': 'median',
    'レース上り4F': 'median',
    'レース上り5F': 'median',
    '中盤ラップ1': 'median',
    '中盤ラップ2': 'median',
    '33ラップ': 'median',
    '最大加速ラップ': 'median',
    'ゴール前ラップ差': 'median',
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング偏差値': 'median',
    'レイティング順位': 'median',
    'レイティング順位分布':'median',
    '追切指数': 'median',
    '追切指数偏差値': 'median',
    '追切指数順位': 'median',
    '追切指数順位分布': 'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'ZI指数偏差値': 'median',
    'ZI指数順位分布': 'median',
    'マイニング': 'median',
    'マイニング偏差値': 'median',
    'マイニング順位': 'median',
    'マイニング順位分布': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング偏差値': 'median',
    '対戦型マイニング順位': 'median',
    '対戦型マイニング順位分布': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '前半3F偏差値': 'median',
    '前半3F順位': 'median',
    '前半3F順位分布': 'median',
    '上り3F': 'median',
    '上り3F偏差値': 'median',
    '上り3F順位': 'median',
    '上り3F順位分布': 'median',
    'Ave-3F': 'median',
    'Ave-3F偏差値': 'median',
    'Ave-3F順位': 'median',
    'Ave-3F順位分布': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数偏差値': 'median',
    'テン指数順位': 'median',
    'テン指数順位分布': 'median',
    '上り指数': 'median',
    '上り指数偏差値': 'median',
    '上り指数順位': 'median',
    '上り指数順位分布': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    'スピード指数': 'median',
    'スピード指数偏差値': 'median',
    'スピード指数順位': 'median',
    'スピード指数順位分布': 'median',
    '総合指数': 'median',
    '総合指数偏差値': 'median',
    '総合指数順位': 'median',
    '総合指数順位分布': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角位置': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

agg_cols4 = {
    '生後日数': 'median',
    '体重': 'median',
    '斤量馬体重比': 'median',
    '前走距離': 'median',
    '前走距離差': 'median',
    'レイティング': 'median',
    'レイティング偏差値': 'median',
    'レイティング順位': 'median',
    'レイティング順位分布':'median',
    'ZI指数': 'median',
    'ZI指数順位': 'median',
    'ZI指数偏差値': 'median',
    'ZI指数順位分布': 'median',
    '追切指数': 'median',
    '追切指数順位': 'median',
    '追切指数偏差値': 'median',
    '追切指数順位分布': 'median',
    'マイニング': 'median',
    'マイニング偏差値': 'median',
    'マイニング順位': 'median',
    'マイニング順位分布': 'median',
    '対戦型マイニング': 'median',
    '対戦型マイニング偏差値': 'median',
    '対戦型マイニング順位': 'median',
    '対戦型マイニング順位分布': 'median',
    'PCI': 'median',
    'RPCI差': 'median',
    'タイムS': 'median',
    '補正走破タイム': 'median',
    '-3Fタイム': 'median',
    '前半3F': 'median',
    '前半3F偏差値': 'median',
    '前半3F順位': 'median',
    '前半3F順位分布': 'median',
    '上り3F': 'median',
    '上り3F偏差値': 'median',
    '上り3F順位': 'median',
    '上り3F順位分布': 'median',
    'Ave-3F': 'median',
    'Ave-3F偏差値': 'median',
    'Ave-3F順位': 'median',
    'Ave-3F順位分布': 'median',
    '-3F差': 'median',
    'テン指数': 'median',
    'テン指数偏差値': 'median',
    'テン指数順位': 'median',
    'テン指数順位分布': 'median',
    '上り指数': 'median',
    '上り指数偏差値': 'median',
    '上り指数順位': 'median',
    '上り指数順位分布': 'median',
    'スピード指数': 'median',
    'スピード指数偏差値': 'median',
    'スピード指数順位': 'median',
    'スピード指数順位分布': 'median',
    '補正タイム': 'median',
    '補9': 'median',
    '総合指数': 'median',
    '総合指数偏差値': 'median',
    '総合指数順位': 'median',
    '総合指数順位分布': 'median',
    'レース強度指数': 'median',
    '通過1': 'median',
    '通過2': 'median',
    '通過3': 'median',
    '通過4': 'median',
    '初角位置': 'median',
    '初角_4角差': 'median',
    '4角_入線順位差': 'median',
    'サイドポジション平均': 'median'
}

# --- 基準タイム算出処理2 ---
# 1勝クラス・2勝クラスのみ対象
base_df1 = merged_df1[merged_df1['クラスコード'].isin([23, 43])].copy()

# 1着～3着と1着馬のデータ抽出
df_tops1 = base_df1[base_df1['入線順位'].between(1, 3)].copy()
df_1st1 = base_df1[base_df1['入線順位'] == 1].copy()

# クラス分類の統一
df_tops1.loc[:, 'クラス分類'] = '1勝・2勝クラス'
df_1st1.loc[:, 'クラス分類'] = '1勝・2勝クラス'

# --- 集計処理 ---
base_time_tops1 = df_tops1.groupby(group_cols1, dropna=False).agg(agg_cols3).reset_index()
base_time_1st1 = df_1st1.groupby(group_cols1, dropna=False).agg(agg_cols4).reset_index()
base_time_1st1 = base_time_1st1.rename(columns={col: f"{col}_1着" for col in agg_cols4.keys()})

# --- 結合 ---
base_time_df1 = pd.merge(base_time_tops1, base_time_1st1, on=group_cols1, how="left")

# --- 距離係数（速度換算） ---
base_time_df1['距離係数'] = (1 / base_time_df1['タイムS']) * 100

# --- 基準タイム算出処理2 ---
df_tops2 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
df_1st2 = merged_df1[merged_df1['入線順位'] == 1].copy()

base_time_tops2 = df_tops2.groupby(group_cols2).agg(agg_cols3).reset_index()
base_time_1st2 = df_1st2.groupby(group_cols2).agg(agg_cols4).reset_index()
base_time_1st2 = base_time_1st2.rename(columns={col: f"{col}_1着" for col in agg_cols4.keys()})
base_time_df2 = pd.merge(base_time_tops2, base_time_1st2, on=group_cols2, how="left")

# --- 距離係数（速度換算） ---
base_time_df2['距離係数'] = (1 / base_time_df2['タイムS']) * 100

#---ステップ４：成績データへレースレベル指数の追加
# 再集計した基準DFを suffix 付きで準備
std1_pre = prepare_ref_df(base_time_df1, merge_key1, '_stdtime1_new')
std2_pre = prepare_ref_df(base_time_df2, merge_key2, '_stdtime2_new')

# merged_df1にマージ
merged_df1 = pd.merge(merged_df1, std1_pre, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, std2_pre, on=merge_key2, how='left')

# 不要なデータフレームのクリア
del std1_pre, std2_pre

# ベース値（Std2優先 → なければ Std1）
rating_base = merged_df1['レイティング_stdtime2_new'].fillna(
    merged_df1['レイティング_stdtime1_new']
)
speed_base = merged_df1['総合指数_stdtime2_new'].fillna(
    merged_df1['総合指数_stdtime1_new']
)
strength_base = merged_df1['レース強度指数_stdtime2_new'].fillna(
    merged_df1['レース強度指数_stdtime1_new']
)

# 差分（NaN は 0 扱い）
rating_diff = (merged_df1['レイティング平均値'] - rating_base).fillna(0)
speed_diff  = (merged_df1['Top3総合指数'] - speed_base).fillna(0)
strength_diff = (merged_df1['レース強度指数'] - strength_base).fillna(0)

# レースレベル指数
merged_df1['レースレベル指数'] = (rating_diff + speed_diff + strength_diff + 100).round(1)

#---ステップ５：各指数の差分を成績データへ追加
# Top3総合指数と総合指数の差分を成績データへ追加
merged_df1['Top3総合指数差分'] = merged_df1['総合指数'] - merged_df1['Top3総合指数']

# レイティング平均値とレイティングの差分を追加
merged_df1['平均レイティング差分'] = merged_df1['レイティング'] - merged_df1['レイティング平均値']

# レース強度指数：基準タイム１差分
merged_df1['レース強度指数上位差分']=merged_df1['レース強度指数']-merged_df1['レース強度指数_stdtime1_new'].fillna(merged_df1['レース強度指数_stdtime2_new'])

# PCI3：基準タイム１差分
merged_df1['PCI3差分']=merged_df1['PCI3']-merged_df1['PCI3_stdtime1_new'].fillna(merged_df1['PCI3_stdtime2_new'])

# PCI：基準タイム１差分
merged_df1['PCI上位差分']=merged_df1['PCI']-merged_df1['PCI_stdtime1_new'].fillna(merged_df1['PCI_stdtime2_new'])
merged_df1['PCI勝馬差分']=merged_df1['PCI']-merged_df1['PCI_1着_stdtime1_new'].fillna(merged_df1['PCI_1着_stdtime2_new'])

# 生後日数：基準タイム２差分
merged_df1['生後日数上位差分']=merged_df1['生後日数']-merged_df1['生後日数_stdtime2_new'].fillna(merged_df1['生後日数_stdtime1_new'])
merged_df1['生後日数勝馬差分']=merged_df1['生後日数']-merged_df1['生後日数_1着_stdtime2_new'].fillna(merged_df1['生後日数_1着_stdtime1_new'])

# 馬体重：基準タイム２差分
merged_df1['体重上位差分']=merged_df1['体重']-merged_df1['体重_stdtime2_new'].fillna(merged_df1['体重_stdtime1_new'])
merged_df1['体重勝馬差分']=merged_df1['体重']-merged_df1['体重_1着_stdtime2_new'].fillna(merged_df1['体重_1着_stdtime1_new'])

# 馬体重斤量比：基準タイム２差分
merged_df1['斤量馬体重比上位差分']=merged_df1['斤量馬体重比']-merged_df1['斤量馬体重比_stdtime2_new'].fillna(merged_df1['斤量馬体重比_stdtime1_new'])
merged_df1['斤量馬体重比勝馬差分']=merged_df1['斤量馬体重比']-merged_df1['斤量馬体重比_1着_stdtime2_new'].fillna(merged_df1['斤量馬体重比_1着_stdtime1_new'])

# 前走距離：基準タイム１差分
merged_df1['前走距離上位差分']=merged_df1['前走距離']-merged_df1['前走距離_stdtime1_new'].fillna(merged_df1['前走距離_stdtime2_new'])
merged_df1['前走距離勝馬差分']=merged_df1['前走距離']-merged_df1['前走距離_1着_stdtime1_new'].fillna(merged_df1['前走距離_1着_stdtime2_new'])

# 前走距離差：基準タイム１差分
merged_df1['前走距離差上位差分']=merged_df1['前走距離差']-merged_df1['前走距離差_stdtime1_new'].fillna(merged_df1['前走距離差_stdtime2_new'])
merged_df1['前走距離差勝馬差分']=merged_df1['前走距離差']-merged_df1['前走距離差_1着_stdtime1_new'].fillna(merged_df1['前走距離差_1着_stdtime2_new'])

# レイティング：基準タイム１差分
merged_df1['レイティング上位差分']=merged_df1['レイティング']-merged_df1['レイティング_stdtime1_new'].fillna(merged_df1['レイティング_stdtime2_new'])
merged_df1['レイティング勝馬差分']=merged_df1['レイティング']-merged_df1['レイティング_1着_stdtime1_new'].fillna(merged_df1['レイティング_1着_stdtime2_new'])

# レイティング偏差値：基準タイム１差分
merged_df1['レイティング偏差値上位差分']=merged_df1['レイティング偏差値']-merged_df1['レイティング偏差値_stdtime1_new'].fillna(merged_df1['レイティング偏差値_stdtime2_new'])
merged_df1['レイティング偏差値勝馬差分']=merged_df1['レイティング偏差値']-merged_df1['レイティング偏差値_1着_stdtime1_new'].fillna(merged_df1['レイティング偏差値_1着_stdtime2_new'])

# レイティング順位分布：基準タイム１差分
merged_df1['レイティング順位分布上位差分']=merged_df1['レイティング順位分布']-merged_df1['レイティング順位分布_stdtime1_new'].fillna(merged_df1['レイティング順位分布_stdtime2_new'])
merged_df1['レイティング順位分布勝馬差分']=merged_df1['レイティング順位分布']-merged_df1['レイティング順位分布_1着_stdtime1_new'].fillna(merged_df1['レイティング順位分布_1着_stdtime2_new'])

# ZI指数：基準タイム１差分
merged_df1['ZI指数上位差分']=merged_df1['ZI指数']-merged_df1['ZI指数_stdtime1_new'].fillna(merged_df1['ZI指数_stdtime2_new'])
merged_df1['ZI指数勝馬差分']=merged_df1['ZI指数']-merged_df1['ZI指数_1着_stdtime1_new'].fillna(merged_df1['ZI指数_1着_stdtime2_new'])

# ZI指数偏差値：基準タイム１差分
merged_df1['ZI指数偏差値上位差分']=merged_df1['ZI指数偏差値']-merged_df1['ZI指数偏差値_stdtime1_new'].fillna(merged_df1['ZI指数偏差値_stdtime2_new'])
merged_df1['ZI指数偏差値勝馬差分']=merged_df1['ZI指数偏差値']-merged_df1['ZI指数偏差値_1着_stdtime1_new'].fillna(merged_df1['ZI指数偏差値_1着_stdtime2_new'])

# ZI指数順位分布：基準タイム１差分
merged_df1['ZI指数順位分布上位差分']=merged_df1['ZI指数順位分布']-merged_df1['ZI指数順位分布_stdtime1_new'].fillna(merged_df1['ZI指数順位分布_stdtime2_new'])
merged_df1['ZI指数順位分布勝馬差分']=merged_df1['ZI指数順位分布']-merged_df1['ZI指数順位分布_1着_stdtime1_new'].fillna(merged_df1['ZI指数順位分布_1着_stdtime2_new'])

# 追切指数：基準タイム１差分
merged_df1['追切指数上位差分']=merged_df1['追切指数']-merged_df1['追切指数_stdtime1_new'].fillna(merged_df1['追切指数_stdtime2_new'])
merged_df1['追切指数勝馬差分']=merged_df1['追切指数']-merged_df1['追切指数_1着_stdtime1_new'].fillna(merged_df1['追切指数_1着_stdtime2_new'])

# 追切指数偏差値：基準タイム１差分
merged_df1['追切指数偏差値上位差分']=merged_df1['追切指数偏差値']-merged_df1['追切指数偏差値_stdtime1_new'].fillna(merged_df1['追切指数偏差値_stdtime2_new'])
merged_df1['追切指数偏差値勝馬差分']=merged_df1['追切指数偏差値']-merged_df1['追切指数偏差値_1着_stdtime1_new'].fillna(merged_df1['追切指数偏差値_1着_stdtime2_new'])

# 追切指数順位分布：基準タイム１差分
merged_df1['追切指数順位分布上位差分']=merged_df1['追切指数順位分布']-merged_df1['追切指数順位分布_stdtime1_new'].fillna(merged_df1['追切指数順位分布_stdtime2_new'])
merged_df1['追切指数順位分布勝馬差分']=merged_df1['追切指数順位分布']-merged_df1['追切指数順位分布_1着_stdtime1_new'].fillna(merged_df1['追切指数順位分布_1着_stdtime2_new'])

# マイニング：基準タイム１差分
merged_df1['マイニング上位差分']=merged_df1['マイニング']-merged_df1['マイニング_stdtime1_new'].fillna(merged_df1['マイニング_stdtime2_new'])
merged_df1['マイニング勝馬差分']=merged_df1['マイニング']-merged_df1['マイニング_1着_stdtime1_new'].fillna(merged_df1['マイニング_1着_stdtime2_new'])

# マイニング偏差値：基準タイム１差分
merged_df1['マイニング偏差値上位差分']=merged_df1['マイニング偏差値']-merged_df1['マイニング偏差値_stdtime1_new'].fillna(merged_df1['マイニング偏差値_stdtime2_new'])
merged_df1['マイニング偏差値勝馬差分']=merged_df1['マイニング偏差値']-merged_df1['マイニング偏差値_1着_stdtime1_new'].fillna(merged_df1['マイニング偏差値_1着_stdtime2_new'])

# マイニング順位分布：基準タイム１差分
merged_df1['マイニング順位分布上位差分']=merged_df1['マイニング順位分布']-merged_df1['マイニング順位分布_stdtime1_new'].fillna(merged_df1['マイニング順位分布_stdtime2_new'])
merged_df1['マイニング順位分布勝馬差分']=merged_df1['マイニング順位分布']-merged_df1['マイニング順位分布_1着_stdtime1_new'].fillna(merged_df1['マイニング順位分布_1着_stdtime2_new'])

# 対戦型マイニング：基準タイム１差分
merged_df1['対戦型マイニング上位差分']=merged_df1['対戦型マイニング']-merged_df1['対戦型マイニング_stdtime1_new'].fillna(merged_df1['対戦型マイニング_stdtime2_new'])
merged_df1['対戦型マイニング勝馬差分']=merged_df1['対戦型マイニング']-merged_df1['対戦型マイニング_1着_stdtime1_new'].fillna(merged_df1['対戦型マイニング_1着_stdtime2_new'])

# 対戦型マイニング偏差値：基準タイム１差分
merged_df1['対戦型マイニング偏差値上位差分']=merged_df1['対戦型マイニング偏差値']-merged_df1['対戦型マイニング偏差値_stdtime1_new'].fillna(merged_df1['対戦型マイニング偏差値_stdtime2_new'])
merged_df1['対戦型マイニング偏差値勝馬差分']=merged_df1['対戦型マイニング偏差値']-merged_df1['対戦型マイニング偏差値_1着_stdtime1_new'].fillna(merged_df1['対戦型マイニング偏差値_1着_stdtime2_new'])

# 対戦型マイニング順位分布：基準タイム１差分
merged_df1['対戦型マイニング順位分布上位差分']=merged_df1['対戦型マイニング順位分布']-merged_df1['対戦型マイニング順位分布_stdtime1_new'].fillna(merged_df1['対戦型マイニング順位分布_stdtime2_new'])
merged_df1['対戦型マイニング順位分布勝馬差分']=merged_df1['対戦型マイニング順位分布']-merged_df1['対戦型マイニング順位分布_1着_stdtime1_new'].fillna(merged_df1['対戦型マイニング順位分布_1着_stdtime2_new'])

# タイムS：基準タイム１差分
merged_df1['タイムS上位差分']=merged_df1['タイムS']-merged_df1['タイムS_stdtime1_new'].fillna(merged_df1['タイムS_stdtime2_new'])
merged_df1['タイムS勝馬差分']=merged_df1['タイムS']-merged_df1['タイムS_1着_stdtime1_new'].fillna(merged_df1['タイムS_1着_stdtime2_new'])

# 補正走破タイム：基準タイム１差分
merged_df1['補正走破タイム上位差分']=merged_df1['補正走破タイム']-merged_df1['補正走破タイム_stdtime1_new'].fillna(merged_df1['補正走破タイム_stdtime2_new'])
merged_df1['補正走破タイム勝馬差分']=merged_df1['補正走破タイム']-merged_df1['補正走破タイム_1着_stdtime1_new'].fillna(merged_df1['補正走破タイム_1着_stdtime2_new'])

# -3Fタイム：基準タイム１差分
merged_df1['-3Fタイム上位差分']=merged_df1['-3Fタイム']-merged_df1['-3Fタイム_stdtime1_new'].fillna(merged_df1['-3Fタイム_stdtime2_new'])
merged_df1['-3Fタイム勝馬差分']=merged_df1['-3Fタイム']-merged_df1['-3Fタイム_1着_stdtime1_new'].fillna(merged_df1['-3Fタイム_1着_stdtime2_new'])

# 前半3F：基準タイム１差分
merged_df1['前半3F上位差分']=merged_df1['前半3F']-merged_df1['前半3F_stdtime1_new'].fillna(merged_df1['前半3F_stdtime2_new'])
merged_df1['前半3F勝馬差分']=merged_df1['前半3F']-merged_df1['前半3F_1着_stdtime1_new'].fillna(merged_df1['前半3F_1着_stdtime2_new'])

# 前半3F偏差値：基準タイム１差分
merged_df1['前半3F偏差値上位差分']=merged_df1['前半3F偏差値']-merged_df1['前半3F偏差値_stdtime1_new'].fillna(merged_df1['前半3F偏差値_stdtime2_new'])
merged_df1['前半3F偏差値勝馬差分']=merged_df1['前半3F偏差値']-merged_df1['前半3F偏差値_1着_stdtime1_new'].fillna(merged_df1['前半3F偏差値_1着_stdtime2_new'])

# 前半3F順位：基準タイム１差分
merged_df1['前半3F順位上位差分']=merged_df1['前半3F順位']-merged_df1['前半3F順位_stdtime1_new'].fillna(merged_df1['前半3F順位_stdtime2_new'])
merged_df1['前半3F順位勝馬差分']=merged_df1['前半3F順位']-merged_df1['前半3F順位_1着_stdtime1_new'].fillna(merged_df1['前半3F順位_1着_stdtime2_new'])

# 前半3F順位分布：基準タイム１差分
merged_df1['前半3F順位分布上位差分']=merged_df1['前半3F順位分布']-merged_df1['前半3F順位分布_stdtime1_new'].fillna(merged_df1['前半3F順位分布_stdtime2_new'])
merged_df1['前半3F順位分布勝馬差分']=merged_df1['前半3F順位分布']-merged_df1['前半3F順位分布_1着_stdtime1_new'].fillna(merged_df1['前半3F順位分布_1着_stdtime2_new'])

# 上り3F：基準タイム１差分
merged_df1['上り3F上位差分']=merged_df1['上り3F']-merged_df1['上り3F_stdtime1_new'].fillna(merged_df1['上り3F_stdtime2_new'])
merged_df1['上り3F勝馬差分']=merged_df1['上り3F']-merged_df1['上り3F_1着_stdtime1_new'].fillna(merged_df1['上り3F_1着_stdtime2_new'])

# 上り3F偏差値：基準タイム１差分
merged_df1['上り3F偏差値上位差分']=merged_df1['上り3F偏差値']-merged_df1['上り3F偏差値_stdtime1_new'].fillna(merged_df1['上り3F偏差値_stdtime2_new'])
merged_df1['上り3F偏差値勝馬差分']=merged_df1['上り3F偏差値']-merged_df1['上り3F偏差値_1着_stdtime1_new'].fillna(merged_df1['上り3F偏差値_1着_stdtime2_new'])

# 上り3F順位：基準タイム１差分
merged_df1['上り3F順位上位差分']=merged_df1['上り3F順位']-merged_df1['上り3F順位_stdtime1_new'].fillna(merged_df1['上り3F順位_stdtime2_new'])
merged_df1['上り3F順位勝馬差分']=merged_df1['上り3F順位']-merged_df1['上り3F順位_1着_stdtime1_new'].fillna(merged_df1['上り3F順位_1着_stdtime2_new'])

# 上り3F順位分布：基準タイム１差分
merged_df1['上り3F順位分布上位差分']=merged_df1['上り3F順位分布']-merged_df1['上り3F順位分布_stdtime1_new'].fillna(merged_df1['上り3F順位分布_stdtime2_new'])
merged_df1['上り3F順位分布勝馬差分']=merged_df1['上り3F順位分布']-merged_df1['上り3F順位分布_1着_stdtime1_new'].fillna(merged_df1['上り3F順位分布_1着_stdtime2_new'])

# Ave-3F：基準タイム１差分
merged_df1['Ave-3F上位差分']=merged_df1['Ave-3F']-merged_df1['Ave-3F_stdtime1_new'].fillna(merged_df1['Ave-3F_stdtime2_new'])
merged_df1['Ave-3F勝馬差分']=merged_df1['Ave-3F']-merged_df1['Ave-3F_1着_stdtime1_new'].fillna(merged_df1['Ave-3F_1着_stdtime2_new'])

# Ave-3F偏差値：基準タイム１差分
merged_df1['Ave-3F偏差値上位差分']=merged_df1['Ave-3F偏差値']-merged_df1['Ave-3F偏差値_stdtime1_new'].fillna(merged_df1['Ave-3F偏差値_stdtime2_new'])
merged_df1['Ave-3F偏差値勝馬差分']=merged_df1['Ave-3F偏差値']-merged_df1['Ave-3F偏差値_1着_stdtime1_new'].fillna(merged_df1['Ave-3F偏差値_1着_stdtime2_new'])

# Ave-3F順位：基準タイム１差分
merged_df1['Ave-3F順位上位差分']=merged_df1['Ave-3F順位']-merged_df1['Ave-3F順位_stdtime1_new'].fillna(merged_df1['Ave-3F順位_stdtime2_new'])
merged_df1['Ave-3F順位勝馬差分']=merged_df1['Ave-3F順位']-merged_df1['Ave-3F順位_1着_stdtime1_new'].fillna(merged_df1['Ave-3F順位_1着_stdtime2_new'])

# Ave-3F順位分布：基準タイム１差分
merged_df1['Ave-3F順位分布上位差分']=merged_df1['Ave-3F順位分布']-merged_df1['Ave-3F順位分布_stdtime1_new'].fillna(merged_df1['Ave-3F順位分布_stdtime2_new'])
merged_df1['Ave-3F順位分布勝馬差分']=merged_df1['Ave-3F順位分布']-merged_df1['Ave-3F順位分布_1着_stdtime1_new'].fillna(merged_df1['Ave-3F順位分布_1着_stdtime2_new'])

# -3F差：基準タイム１差分
merged_df1['-3F差上位差分']=merged_df1['-3F差']-merged_df1['-3F差_stdtime1_new'].fillna(merged_df1['-3F差_stdtime2_new'])
merged_df1['-3F差勝馬差分']=merged_df1['-3F差']-merged_df1['-3F差_1着_stdtime1_new'].fillna(merged_df1['-3F差_1着_stdtime2_new'])

# テン指数：基準タイム１差分
merged_df1['テン指数上位差分']=merged_df1['テン指数']-merged_df1['テン指数_stdtime1_new'].fillna(merged_df1['テン指数_stdtime2_new'])
merged_df1['テン指数勝馬差分']=merged_df1['テン指数']-merged_df1['テン指数_1着_stdtime1_new'].fillna(merged_df1['テン指数_1着_stdtime2_new'])

# テン指数偏差値：基準タイム１差分
merged_df1['テン指数偏差値上位差分']=merged_df1['テン指数偏差値']-merged_df1['テン指数偏差値_stdtime1_new'].fillna(merged_df1['テン指数偏差値_stdtime2_new'])
merged_df1['テン指数偏差値勝馬差分']=merged_df1['テン指数偏差値']-merged_df1['テン指数偏差値_1着_stdtime1_new'].fillna(merged_df1['テン指数偏差値_1着_stdtime2_new'])

# テン指数順位：基準タイム１差分
merged_df1['テン指数順位上位差分']=merged_df1['テン指数順位']-merged_df1['テン指数順位_stdtime1_new'].fillna(merged_df1['テン指数順位_stdtime2_new'])
merged_df1['テン指数順位勝馬差分']=merged_df1['テン指数順位']-merged_df1['テン指数順位_1着_stdtime1_new'].fillna(merged_df1['テン指数順位_1着_stdtime2_new'])

# テン指数順位分布：基準タイム１差分
merged_df1['テン指数順位分布上位差分']=merged_df1['テン指数順位分布']-merged_df1['テン指数順位分布_stdtime1_new'].fillna(merged_df1['テン指数順位分布_stdtime2_new'])
merged_df1['テン指数順位分布勝馬差分']=merged_df1['テン指数順位分布']-merged_df1['テン指数順位分布_1着_stdtime1_new'].fillna(merged_df1['テン指数順位分布_1着_stdtime2_new'])

# 上り指数：基準タイム１差分
merged_df1['上り指数上位差分']=merged_df1['上り指数']-merged_df1['上り指数_stdtime1_new'].fillna(merged_df1['上り指数_stdtime2_new'])
merged_df1['上り指数勝馬差分']=merged_df1['上り指数']-merged_df1['上り指数_1着_stdtime1_new'].fillna(merged_df1['上り指数_1着_stdtime2_new'])

# 上り指数偏差値：基準タイム１差分
merged_df1['上り指数偏差値上位差分']=merged_df1['上り指数偏差値']-merged_df1['上り指数偏差値_stdtime1_new'].fillna(merged_df1['上り指数偏差値_stdtime2_new'])
merged_df1['上り指数偏差値勝馬差分']=merged_df1['上り指数偏差値']-merged_df1['上り指数偏差値_1着_stdtime1_new'].fillna(merged_df1['上り指数偏差値_1着_stdtime2_new'])

# 上り指数順位：基準タイム１差分
merged_df1['上り指数順位上位差分']=merged_df1['上り指数順位']-merged_df1['上り指数順位_stdtime1_new'].fillna(merged_df1['上り指数順位_stdtime2_new'])
merged_df1['上り指数順位勝馬差分']=merged_df1['上り指数順位']-merged_df1['上り指数順位_1着_stdtime1_new'].fillna(merged_df1['上り指数順位_1着_stdtime2_new'])

# 上り指数順位分布：基準タイム１差分
merged_df1['上り指数順位分布上位差分']=merged_df1['上り指数順位分布']-merged_df1['上り指数順位分布_stdtime1_new'].fillna(merged_df1['上り指数順位分布_stdtime2_new'])
merged_df1['上り指数順位分布勝馬差分']=merged_df1['上り指数順位分布']-merged_df1['上り指数順位分布_1着_stdtime1_new'].fillna(merged_df1['上り指数順位分布_1着_stdtime2_new'])

# スピード指数：基準タイム１差分
merged_df1['スピード指数上位差分']=merged_df1['スピード指数']-merged_df1['スピード指数_stdtime1_new'].fillna(merged_df1['スピード指数_stdtime2_new'])
merged_df1['スピード指数勝馬差分']=merged_df1['スピード指数']-merged_df1['スピード指数_1着_stdtime1_new'].fillna(merged_df1['スピード指数_1着_stdtime2_new'])

# スピード指数偏差値：基準タイム１差分
merged_df1['スピード指数偏差値上位差分']=merged_df1['スピード指数偏差値']-merged_df1['スピード指数偏差値_stdtime1_new'].fillna(merged_df1['スピード指数偏差値_stdtime2_new'])
merged_df1['スピード指数偏差値勝馬差分']=merged_df1['スピード指数偏差値']-merged_df1['スピード指数偏差値_1着_stdtime1_new'].fillna(merged_df1['スピード指数偏差値_1着_stdtime2_new'])

# スピード指数順位：基準タイム１差分
merged_df1['スピード指数順位上位差分']=merged_df1['スピード指数順位']-merged_df1['スピード指数順位_stdtime1_new'].fillna(merged_df1['スピード指数順位_stdtime2_new'])
merged_df1['スピード指数順位勝馬差分']=merged_df1['スピード指数順位']-merged_df1['スピード指数順位_1着_stdtime1_new'].fillna(merged_df1['スピード指数順位_1着_stdtime2_new'])

# スピード指数順位分布：基準タイム１差分
merged_df1['スピード指数順位分布上位差分']=merged_df1['スピード指数順位分布']-merged_df1['スピード指数順位分布_stdtime1_new'].fillna(merged_df1['スピード指数順位分布_stdtime2_new'])
merged_df1['スピード指数順位分布勝馬差分']=merged_df1['スピード指数順位分布']-merged_df1['スピード指数順位分布_1着_stdtime1_new'].fillna(merged_df1['スピード指数順位分布_1着_stdtime2_new'])

# 総合指数：基準タイム１差分
merged_df1['総合指数上位差分']=merged_df1['総合指数']-merged_df1['総合指数_stdtime1_new'].fillna(merged_df1['総合指数_stdtime2_new'])
merged_df1['総合指数勝馬差分']=merged_df1['総合指数']-merged_df1['総合指数_1着_stdtime1_new'].fillna(merged_df1['総合指数_1着_stdtime2_new'])

# 総合指数偏差値：基準タイム１差分
merged_df1['総合指数偏差値上位差分']=merged_df1['総合指数偏差値']-merged_df1['総合指数偏差値_stdtime1_new'].fillna(merged_df1['総合指数偏差値_stdtime2_new'])
merged_df1['総合指数偏差値勝馬差分']=merged_df1['総合指数偏差値']-merged_df1['総合指数偏差値_1着_stdtime1_new'].fillna(merged_df1['総合指数偏差値_1着_stdtime2_new'])

# 総合指数順位：基準タイム１差分
merged_df1['総合指数順位上位差分']=merged_df1['総合指数順位']-merged_df1['総合指数順位_stdtime1_new'].fillna(merged_df1['総合指数順位_stdtime2_new'])
merged_df1['総合指数順位勝馬差分']=merged_df1['総合指数順位']-merged_df1['総合指数順位_1着_stdtime1_new'].fillna(merged_df1['総合指数順位_1着_stdtime2_new'])

# 総合指数順位分布：基準タイム１差分
merged_df1['総合指数順位分布上位差分']=merged_df1['総合指数順位分布']-merged_df1['総合指数順位分布_stdtime1_new'].fillna(merged_df1['総合指数順位分布_stdtime2_new'])
merged_df1['総合指数順位分布勝馬差分']=merged_df1['総合指数順位分布']-merged_df1['総合指数順位分布_1着_stdtime1_new'].fillna(merged_df1['総合指数順位分布_1着_stdtime2_new'])

# PCI：基準タイム１差分
merged_df1['PCI上位差分']=merged_df1['PCI']-merged_df1['PCI_stdtime1_new'].fillna(merged_df1['PCI_stdtime2_new'])
merged_df1['PCI勝馬差分']=merged_df1['PCI']-merged_df1['PCI_1着_stdtime1_new'].fillna(merged_df1['PCI_1着_stdtime2_new'])

# 初角_4角差：基準タイム１差分
merged_df1['初角_4角差上位差分']=merged_df1['初角_4角差']-merged_df1['初角_4角差_stdtime1_new'].fillna(merged_df1['初角_4角差_stdtime2_new'])
merged_df1['初角_4角差勝馬差分']=merged_df1['初角_4角差']-merged_df1['初角_4角差_1着_stdtime1_new'].fillna(merged_df1['初角_4角差_1着_stdtime2_new'])

# 4角_入線順位差：基準タイム１差分
merged_df1['4角_入線順位差上位差分']=merged_df1['4角_入線順位差']-merged_df1['4角_入線順位差_stdtime1_new'].fillna(merged_df1['4角_入線順位差_stdtime2_new'])
merged_df1['4角_入線順位差勝馬差分']=merged_df1['4角_入線順位差']-merged_df1['4角_入線順位差_1着_stdtime1_new'].fillna(merged_df1['4角_入線順位差_1着_stdtime2_new'])

#---ステップ６：レースレベル基準の作成
# グループ化キー
group_cols3 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類']

# --- 基準レースレベル算出処理 ---
racelevel_df = (
    merged_df1
      .dropna(subset=['レースレベル指数'])  # 欠損を除外
      .groupby(group_cols3, dropna=False)['レースレベル指数']
      .agg(['mean', 'std'])  # mean と std を同時に集計
      .reset_index()
).copy()

# 近似Zスコア用の基準値を計算
racelevel_df['RL±0'] = racelevel_df['mean'].round(2)
racelevel_df['RL+1'] = (racelevel_df['mean'] + racelevel_df['std']).round(2)
racelevel_df['RL-1'] = (racelevel_df['mean'] - racelevel_df['std']).round(2)
racelevel_df['RL+2'] = (racelevel_df['mean'] + 2 * racelevel_df['std']).round(2)
racelevel_df['RL-2'] = (racelevel_df['mean'] - 2 * racelevel_df['std']).round(2)

# 必要なカラムだけ残す
racelevel_df = racelevel_df[
    group_cols3 + ['RL-2','RL-1','RL±0','RL+1','RL+2','mean','std']
].copy()

#---ステップ７：レースレベル判定
# レースレベル基準のデータフレームにサフィックスを付ける
merged_df2 = prepare_ref_df(racelevel_df, group_cols3, '_racelevel')

merged_df1 = pd.merge(merged_df1, merged_df2, on=group_cols3, how='left')

# 不要なデータフレームのクリア
del merged_df2

# 差分
rl_m2 = (merged_df1['レースレベル指数'] - merged_df1['RL-2_racelevel']).abs()
rl_m1 = (merged_df1['レースレベル指数'] - merged_df1['RL-1_racelevel']).abs()
rl_0  = (merged_df1['レースレベル指数'] - merged_df1['RL±0_racelevel']).abs()
rl_p1 = (merged_df1['レースレベル指数'] - merged_df1['RL+1_racelevel']).abs()
rl_p2 = (merged_df1['レースレベル指数'] - merged_df1['RL+2_racelevel']).abs()

# 5本を横に並べて「最小の列名」を取る（行ごと）
diff_df = pd.concat([rl_m2, rl_m1, rl_0, rl_p1, rl_p2], axis=1)
diff_df.columns = [-2, -1, 0, 1, 2]  # そのまま判定値にする

# 「基準が無い行」判定（5本すべてNaN）
no_ref = diff_df.isna().all(axis=1)

# idxminを安定させるため NaN は無限大扱いにして最小を取る
band = diff_df.fillna(np.inf).idxmin(axis=1)

# レースレベル判定 列を追加
insert_pos = merged_df1.columns.get_loc('レースレベル指数') + 1
merged_df1.insert(insert_pos, 'レースレベル判定', band.where(~no_ref, np.nan).astype('Int64'))

# レースレベル評価 列を追加
band_to_grade = {2: 'A', 1: 'B', 0: 'C', -1: 'D', -2: 'E'}
insert_pos = merged_df1.columns.get_loc('レースレベル判定') + 1
merged_df1.insert(insert_pos, 'レースレベル評価', merged_df1['レースレベル判定'].map(band_to_grade))

# レース印３ 列を追加
rank_to_label = {'A': '05A', 'B': '07B', 'C': '01C', 'D': '00D', 'E': '03E'}
merged_df1['レース印３'] = merged_df1['レースレベル評価'].replace(rank_to_label)

# サフィックス付き列を削除
# 削除するサフィックスを列挙
suffixes_to_drop = ('_stdtime1_new','_stdtime2_new', '_racelevel')

# サフィックスで判定して残す列だけ抜き出し
merged_df1 = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

#---ステップ８：馬の持久力/瞬発力タイプ判定
# merged_df1 をコピーして、1着～3着だけにする
group_cols4 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類']

horsetype_df = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()

horsetype_df= horsetype_df[['target_raceid', '血統登録番号','PCI3', 'PCI'] + group_cols4].copy()

# 基準タイムファイル１から基準PCI3を取ってマージ
std1_df = base_time_df1[group_cols4 + ['PCI3']].copy()
std1_df = std1_df.rename(columns={'PCI3': 'PCI3_stdtime1'})

horsetype_df = pd.merge(horsetype_df, std1_df, on=group_cols4, how='left')

# 不要なデータフレームのクリア
del std1_df

# レース性質（基準比）と、馬のレース内差分を合算
horsetype_df['基準PCI3差分'] = horsetype_df['PCI3'] - horsetype_df['PCI3_stdtime1']
horsetype_df['レースPCI3差分'] = horsetype_df['PCI'] - horsetype_df['PCI3']
horsetype_df['PCI判定スコア'] = horsetype_df['基準PCI3差分'] + horsetype_df['レースPCI3差分']

# 馬ごとに PCI3差分 の中央値を取る
horsetype_df = horsetype_df.groupby('血統登録番号', as_index=False)['PCI判定スコア'].median()

# 'Ｃ' 列を追加して判定（マイナス=持、プラス=瞬）
horsetype_df['Ｃ'] = ''
horsetype_df.loc[horsetype_df['PCI判定スコア'] < 0, 'Ｃ'] = '0'
horsetype_df.loc[horsetype_df['PCI判定スコア'] > 0, 'Ｃ'] = '1'

# 成績データへ馬タイプを代入する
c_map = horsetype_df.set_index('血統登録番号')['Ｃ']
merged_df1['Ｃ'] = merged_df1['血統登録番号'].map(c_map)

# 今日の日付を yymmdd で作る
today_yymmdd = datetime.now().strftime("%y%m%d")

# 判定済み horsetype_df（= 血統登録番号 + Ｃ）に馬名を付ける
horse_name_df = (
    merged_df1[['血統登録番号', '馬名']]
    .dropna(subset=['馬名'])
    .drop_duplicates(subset=['血統登録番号'], keep='last')
)

checkhorse_df = pd.merge(horsetype_df, horse_name_df, on='血統登録番号', how='left')

# 不要なデータフレームのクリア
del horsetype_df
del horse_name_df

# ★「0/1が入った馬だけ」出力（中央値0や判定不能は除外）
checkhorse_df = checkhorse_df[checkhorse_df['Ｃ'].isin(['0', '1'])].copy()

# TARGET仕様の列名に変換
checkhorse_df = checkhorse_df.rename(columns={'Ｃ': 'タイプ'})

# 登録日
checkhorse_df['登録日'] = today_yymmdd

# 血統登録番号の整形（UX～なら zfill しない方が安全）
checkhorse_df['血統登録番号'] = checkhorse_df['血統登録番号'].astype(str).str.strip()

# 列順
checkhorse_df = checkhorse_df[['馬名', 'タイプ', '登録日', '血統登録番号']]

#---ステップ９：csvファイル保存
# 成績データの列の並び替え
header_df = pd.read_csv(header_filepath, header = None, encoding='cp932')
column_list = header_df[0].tolist()
merged_df1 = merged_df1[column_list]

# csvファイル保存
merged_df1.to_csv(output_filepath1, index=False, encoding='cp932')
base_time_df1.to_csv(output_filepath2, index=False, encoding='cp932')
base_time_df2.to_csv(output_filepath3, index=False, encoding='cp932')
pace_medians_df.to_csv(output_filepath4, index=False, encoding='cp932')
lap33_df.to_csv(output_filepath5, index=False, encoding='cp932')
racelevel_df.to_csv(output_filepath6, index=False, encoding='cp932')
checkhorse_df.to_csv(output_filepath7, index=False, encoding='cp932')

# 各指数をインポート用ファイルに加工して保存
# テン指数
imp_df1 = merged_df1[['target_horseid','テン指数']]
# 上り指数
imp_df2 = merged_df1[['target_horseid','上り指数']]
# スピード指数
imp_df3 = merged_df1[['target_horseid','スピード指数']]
# 総合指数
imp_df4 = merged_df1[['target_horseid','総合指数']]
# 33ラップ判定
imp_df5 = merged_df1[['target_raceid','レース印２']].drop_duplicates('target_raceid')
# レースレベル判定
imp_df6 = merged_df1[['target_raceid','レース印３']].drop_duplicates('target_raceid')

# 各指数のマスタをcsv保存
imp_df1.to_csv(output_dir1 + 'First_Score_Master.csv', index=False, encoding='cp932')
imp_df2.to_csv(output_dir2 + 'Spurt_Score_Master.csv', index=False, encoding='cp932')
imp_df3.to_csv(output_dir3 + 'Speed_Score_Master.csv', index=False, encoding='cp932')
imp_df4.to_csv(output_dir4 + 'Total_Score_Master.csv', index=False, encoding='cp932')
imp_df5.to_csv(output_dir5 + '33Lap_Category_Master.csv', index=False, encoding='cp932')
imp_df6.to_csv(output_dir6 + 'Race_Level_Master.csv', index=False, encoding='cp932')

# 年度列を追加（target_raceid先頭4桁が年）
imp_df1['year'] = pd.to_datetime(
    imp_df1['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df2['year'] = pd.to_datetime(
    imp_df2['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df3['year'] = pd.to_datetime(
    imp_df3['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df4['year'] = pd.to_datetime(
    imp_df4['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df5['year'] = pd.to_datetime(
    imp_df5['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

imp_df6['year'] = pd.to_datetime(
    imp_df6['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# 年度別にファイル分割
for year, df in imp_df1.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir1}First_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df2.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir2}Spurt_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df3.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir3}Speed_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df4.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir4}Total_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df5.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir5}33Lap_Category_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in imp_df6.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir6}Race_Level_Master_{year}.csv',
        index=False, encoding='cp932'
    )

# 不要なデータフレームのクリア
del merged_df1
del base_time_df1
del base_time_df2
del pace_medians_df
del lap33_df
del racelevel_df
del imp_df1
del imp_df2
del imp_df3
del imp_df4
del imp_df5
del imp_df6

C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\2686476637.py:68: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(master_file_path, encoding='cp932')
C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\2686476637.py:728: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(remove_outliers)
C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\2686476637.py:1728: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

# 追切指数を算出する
## 2024年分の追切指数を2017~2023年分の基準データで算出

In [3]:
import pandas as pd
import numpy as np
import re
import os
import math

from datetime import datetime
from tkinter import Tk, filedialog

# =========================
# 読み込みファイルパス
# =========================
master_filepath  = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Original\ResultData_2024.csv'

# 基準ファイル
p_course_median  = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_course.csv'
p_class_median   = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_class.csv'
p_course_top20   = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_course_top20.csv'
p_course_top10   = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Training_Data\traning_std_course_top10.csv'

# 出力
output_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\210_Training_Score\Training_Score_Master.csv'
output_dir       = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\210_Training_Score\\'

# =========================
# ファイル選択
# =========================
root = Tk()
root.withdraw()

traning_filepath = filedialog.askopenfilename(
    title="追切CSVファイルを選択してください",
    filetypes=[("CSV Files", "*.csv")]
)
if not traning_filepath:
    print("ファイルが選択されなかったため、処理を終了します。")
    raise SystemExit

# =========================
# 読み込み & 前処理
# =========================
df1 = pd.read_csv(traning_filepath, encoding='cp932').copy()
df2 = pd.read_csv(master_filepath,  encoding='cp932').copy()

def clean_training_data(df):
    df = df.copy()
    mask_invalid = df["日付"].astype(str).str.contains("■|◇", na=False)
    df = df[~mask_invalid].copy()
    check_cols = [c for c in ['8F','7F','6F','5F(4F)','4F(3F)','3F(2F)'] if c in df.columns]
    for c in check_cols:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    is_not_hill = ~df['コース'].astype(str).str.contains('坂', na=False)
    has_10sec_like = np.column_stack([df[c].between(10.0, 19.9) for c in check_cols]).any(axis=1)
    drop_mask = is_not_hill & has_10sec_like
    df = df[~drop_mask].copy()
    return df

df1 = clean_training_data(df1)

replace_map = {'南Ｗ': '美Ｗ', '南Ｄ': '美ダ', '南ダ': '美ダ', '南芝': '美芝'}
df1['コース'] = df1['コース'].replace(replace_map)
df1['馬場状態'] = df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
df1["回り位置"] = pd.to_numeric(df1["回り位置"], errors="coerce")

distance_columns = [col for col in ["8F", "7F", "6F", "5F(4F)", "4F(3F)", "3F(2F)", "1F"] if col in df1.columns]
for col in distance_columns:
    df1[f"{col}_補"] = np.nan

for index, row in df1.iterrows():
    if not pd.isna(row["回り位置"]):
        correction_value = (9 - row["回り位置"]) * 0.1
        valid_times = [col for col in distance_columns if not pd.isna(row[col])]
        if valid_times:
            leftmost_col = valid_times[0]
            leftmost_index = distance_columns.index(leftmost_col)
            df1.at[index, f"{leftmost_col}_補"] = round(row[leftmost_col] + correction_value, 1)
            remaining_cols = distance_columns[leftmost_index + 1:]
            if remaining_cols:
                equal_correction = correction_value / len(remaining_cols)
                for col in remaining_cols:
                    if not pd.isna(row[col]):
                        df1.at[index, f"{col}_補"] = round(row[col] + equal_correction, 1)
        else:
            continue
    else:
        for col in distance_columns:
            if not pd.isna(row[col]):
                df1.at[index, f"{col}_補"] = row[col]

time_columns = [f"{c}_補" for c in distance_columns]

# =========================
# 成績マージ
# =========================
# df2の列をリネームする
# リネームする列名を定義
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid'
}

df2.rename(columns=rename_map, inplace=True)

merge_cols = ['target_horseid', 'トラックコード(JV)', '年齢限定(競走種別コード)', 'クラスコード']
df2 = df2[merge_cols]

def categorize_race_type(race_type):
    if race_type == 11: return 'サラブレッド系2歳'
    elif race_type == 12: return 'サラブレッド系3歳'
    elif race_type >= 13: return 'サラブレッド系3歳以上'
    else: return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15: return '新馬・未勝利'
    elif class_code == 23: return '1勝クラス'
    elif class_code == 43: return '2勝クラス'
    elif class_code == 67: return '3勝クラス'
    elif class_code >= 114: return 'OP・重賞'
    else: return np.nan

df2['年齢限定(競走種別コード)'] = df2['年齢限定(競走種別コード)'].apply(categorize_race_type)
df2['クラスコード'] = df2['クラスコード'].apply(categorize_class_code)

merged_df1 = pd.merge(df1, df2, on='target_horseid', how='inner').copy()
merged_df1['is_saka'] = merged_df1['コース'].astype(str).str.contains('坂', na=False)

# =========================
# 基準テーブル読み込み（MAD対応）
# =========================
def load_and_tag(path, suffix, has_std):
    t = pd.read_csv(path, encoding='cp932').copy()
    rename_map = {}
    for c in time_columns:
        if c in t.columns:
            rename_map[c] = f"{c}_{suffix}"
        
        # 標準偏差ではなくMADを探す
        madc = f"MAD_{c}"
        if has_std and (madc in t.columns):
            rename_map[madc] = f"MAD_{c}_{suffix}"
    return t.rename(columns=rename_map)

df3 = load_and_tag(p_course_median, 'course', True)
df4 = load_and_tag(p_class_median,  'class',  True)
df5 = load_and_tag(p_course_top20, 'course_20', False)
df6 = load_and_tag(p_course_top10, 'course_10', False)

key_course = ['コース','馬場状態']
key_class  = ['年齢限定(競走種別コード)','クラスコード','コース','馬場状態']

def smerge(left, right, keys):
    exist_keys = [k for k in keys if (k in left.columns and k in right.columns)]
    return pd.merge(left, right, on=exist_keys, how='left')

merged_df1 = smerge(merged_df1, df3, key_course)
merged_df1 = smerge(merged_df1, df4, key_class)
merged_df1 = smerge(merged_df1, df5, key_course)
merged_df1 = smerge(merged_df1, df6, key_course)

# =========================
# 偏差値（ロバスト偏差値計算：MAD使用）
# =========================
def dev_vec(v, m, mad):
    # MADを正規分布の標準偏差相当に変換する定数 1.4826
    sigma_est = mad * 1.4826
    
    ok = (~pd.isna(v)) & (~pd.isna(m)) & (~pd.isna(mad)) & (sigma_est != 0)
    
    # タイムは小さい方が良いので (Median - Value)
    # sigma_est が 0 (全員同じタイム等) の場合は偏差値50とする
    return np.where(ok, 50 + 10 * (m - v) / sigma_est, 50)

bases_for_dev = ['course', 'class']

for c in time_columns:
    dev_cols = []
    for b in bases_for_dev:
        mean_col = f"{c}_{b}"
        mad_col  = f"MAD_{c}_{b}"
        colname = f"偏差値_{c}_{b}"
        
        # MADが存在する場合のみ計算
        if mad_col in merged_df1.columns:
            merged_df1[colname] = dev_vec(merged_df1[c], merged_df1[mean_col], merged_df1[mad_col])
            dev_cols.append(colname)

    if dev_cols:
        merged_df1[f"偏差値_統合_{c}"] = merged_df1[dev_cols].mean(axis=1, skipna=True)

# 統合偏差値が存在する列のみでスコア計算
valid_dev_cols = [f"偏差値_統合_{c}" for c in time_columns if f"偏差値_統合_{c}" in merged_df1.columns]
if valid_dev_cols:
    merged_df1['総合偏差値スコア'] = merged_df1[valid_dev_cols].mean(axis=1, skipna=True)
else:
    merged_df1['総合偏差値スコア'] = 50.0

# =========================
# 加点（変更なし）
# =========================
bonus_targets = {'4F': '4F(3F)_補', '2F': '3F(2F)_補', '1F': '1F_補'}

def calculate_bonus(value, mean, is_saka, col_short):
    if pd.isna(value) or pd.isna(mean): return 0.0
    if is_saka:
        if col_short == '4F': return 0.5 if value < mean else 0.0
        if col_short == '2F': return 1.5 if value < mean else 0.0
        if col_short == '1F': return 1.0 if value < mean else 0.0
    else:
        if col_short == '4F': return 0.5 if value < mean else 0.0
        if col_short == '2F': return 1.0 if value < mean else 0.0
        if col_short == '1F': return 1.5 if value < mean else 0.0
    return 0.0

th_sfx_list = ['course_20','course_10']
bonus_cols = []
for col_short, base_col in bonus_targets.items():
    for sfx in th_sfx_list:
        th_col = f"{base_col}_{sfx}"
        if th_col in merged_df1.columns:
            out_col = f"加点_{col_short}_{sfx}"
            merged_df1[out_col] = merged_df1.apply(
                lambda r: calculate_bonus(
                    r.get(base_col, np.nan), r.get(th_col, np.nan),
                    bool(r.get('is_saka', False)), col_short
                ), axis=1
            )
            bonus_cols.append(out_col)

merged_df1['総合加点スコア'] = merged_df1[bonus_cols].sum(axis=1, skipna=True) if bonus_cols else 0.0

# =========================
# 係数・最終指数
# =========================
def rider_coef(x):
    s = '' if pd.isna(x) else str(x)
    if '助手' in s: return 1.0
    if '見習' in s: return 0.8
    return 0.9

def leg_coef(x):
    s = '' if pd.isna(x) else str(x)
    if '馬なり' in s: return 1.1
    if ('Ｇ' in s) or ('G' in s) or ('強' in s): return 1.0
    if '一杯' in s: return 0.8
    if 'ヨレ' in s: return 0.7
    if 'バテ' in s: return 0.6
    return 0.9

merged_df1['騎乗者係数'] = merged_df1['乗り役'].apply(rider_coef)
merged_df1['脚色係数']   = merged_df1['脚色'].apply(leg_coef)

merged_df1['追切指数'] = (
    (merged_df1['総合偏差値スコア'].fillna(50) + merged_df1['総合加点スコア'].fillna(0.0))
    * merged_df1['騎乗者係数'].fillna(0.9)
    * merged_df1['脚色係数'].fillna(0.9)
)

merged_df1['追切指数'] = merged_df1['追切指数'].replace([np.inf, -np.inf], np.nan).fillna(1).round(1)

# 重複排除
merged_unique = merged_df1.loc[merged_df1.groupby('target_horseid')['追切指数'].idxmax()].reset_index(drop=True).copy()
merged_unique = merged_unique[['target_horseid','追切指数']].copy()

del df1,df2,df3,df4,df5,df6,merged_df1

# 保存処理
combined_df = pd.read_csv(output_filepath1, encoding='cp932').copy()
combined_df = pd.concat([combined_df, merged_unique], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset='target_horseid', keep='last')
combined_df.to_csv(output_filepath1, index=False, encoding='cp932')

combined_df['year'] = pd.to_datetime(
    combined_df['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y', errors='coerce'
).dt.year

for year, dfy in combined_df.groupby('year'):
    dfy.drop(columns='year').to_csv(f'{output_dir}Training_Score_Master_{year}.csv', index=False, encoding='cp932')

del combined_df

print("完了：追切指数（MAD・ロバスト偏差値版）を出力しました。")

C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\1771354911.py:43: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(master_filepath,  encoding='cp932').copy()


完了：追切指数（MAD・ロバスト偏差値版）を出力しました。


# 選択した成績データを加工してマスタファイルへマージ
## 2024年分の成績データを2017~2023年の基準タイムファイルを参照して指数を追加
## 2024年分の成績データをマスタファイルへマージ

In [5]:
import pandas as pd
import numpy as np
import re
import os
import math

from datetime import datetime
from tkinter import Tk, filedialog

#---ステップ１：加工する成績データを指定
# ファイルパスの指定
# 成績データファイルのパス
master_filepath = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Processed\Race_Result_Master_2024_tmp.csv'

# 成績データにマージするデータファイルのパス
# 初角位置ファイルのパス
merge_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\70_First_Corner_Position\First_Corner_Position_Master.csv'
# 2角位置ファイルのパス
merge_filepath2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\80_Second_Corner_Position\Second_Corner_Position_Master.csv'
# 3角位置ファイルのパス
merge_filepath3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\90_Third_Corner_Position\Third_Corner_Position_Master.csv'
# 4角位置ファイルのパス
merge_filepath4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\100_Fourth_Corner_Position\Fourth_Corner_Position_Master.csv'
# 上り位置ファイルのパス
merge_filepath5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\110_Spurt_Position\Spurt_Position_Master.csv'
# 馬場指数ファイルのパス
merge_filepath6 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\60_Track_Condition\Track_Condition_Master.csv'
# 前半3Fタイムファイルのパス
merge_filepath7 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\120_First3F_Lap\First3F_Lap_Master.csv'
# 追切指数のファイルパス
merge_filepath8 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\210_Training_Score\Training_Score_Master.csv'

# 参照先のファイルパス
# 基準タイムファイル1の保存先パス
ref_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\StdTime1.csv'
# 基準タイムファイル2の保存先パス
ref_filepath2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\StdTime2.csv'
# ペース係数ファイルの保存先パス
ref_filepath3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\PaceTime.csv'
# 基準33ラップファイルの保存先パス
ref_filepath4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\33Lap.csv'
# レースレベル基準ファイルのパス
ref_filepath5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\30_Index\Times_Data\RaceLevel.csv'

# 保存先パス
# テン指数の保存先
output_filepath1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\150_First_Score\First_Score_Master.csv'
# 上り指数の保存先
output_filepath2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\160_Spurt_Score\Spurt_Score_Master.csv'
# スピード指数の保存先
output_filepath3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\170_Speed_Score\Speed_Score_Master.csv'
# 総合指数の保存先
output_filepath4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\180_Total_Score\Total_Score_Master.csv'
# 33ラップ判定の保存先
output_filepath5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\140_33Lap_Category\33Lap_Category_Master.csv'
# レースレベル判定の保存先
output_filepath6 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\190_Race_Level\Race_Level_Master.csv'
# 馬タイプ分類のファイルパス
output_filepath7 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\220_Horse_Type\Horse_Type_Master.csv'

# テン指数のディレクトリ
output_dir1 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\150_First_Score\\'
# 上り指数のディレクトリ
output_dir2 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\160_Spurt_Score\\'
# スピード指数のディレクトリ
output_dir3 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\170_Speed_Score\\'
# 総合指数のディレクトリ
output_dir4 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\180_Total_Score\\'
# 33ラップ判定のディレクトリ
output_dir5 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\140_33Lap_Category\\'
# レースレベル判定のディレクトリ
output_dir6 = r'G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\00_Import_Data\190_Race_Level\\'

# 列の並び定義ファイルパス
header_filepath = r'G:\マイドライブ\20_HOBBY\20_KEIBA\50_SourceCode\ResultData_Header.csv'

# ファイル選択ダイアログを表示
root = Tk()
root.withdraw()  # メインウィンドウを非表示にする

merged_filepath = filedialog.askopenfilename(title="CSVファイルを選択してください", filetypes=[("CSV Files", "*.csv")])

# ファイルが選択されなかった場合は終了
if not merged_filepath:
    print("ファイルが選択されなかったため、処理を終了します。")
    exit()

# ---ステップ２：マージする成績データの加工
merged_df1 = pd.read_csv(merged_filepath, encoding='cp932').copy()

merged_df2 = pd.read_csv(merge_filepath1, encoding='cp932').copy()
merged_df3 = pd.read_csv(merge_filepath2, encoding='cp932').copy()
merged_df4 = pd.read_csv(merge_filepath3, encoding='cp932').copy()
merged_df5 = pd.read_csv(merge_filepath4, encoding='cp932').copy()
merged_df6 = pd.read_csv(merge_filepath5, encoding='cp932').copy()
merged_df7 = pd.read_csv(merge_filepath6, encoding='cp932').copy()
merged_df8 = pd.read_csv(merge_filepath7, encoding='cp932').copy()
merged_df9 = pd.read_csv(merge_filepath8, encoding='cp932').copy()

#　マージファイルで使用する列だけを定義
merged_df2 = merged_df2[['target_horseid','初角サイドポジション']]
merged_df3 = merged_df3[['target_horseid','2角サイドポジション']]
merged_df4 = merged_df4[['target_horseid','3角サイドポジション']]
merged_df5 = merged_df5[['target_horseid','4角サイドポジション']]
merged_df6 = merged_df6[['target_horseid','4角位置']]
merged_df7 = merged_df7[['target_raceid','馬場指数']]
merged_df8 = merged_df8[['target_horseid','前半3F']]
merged_df9 = merged_df9[['target_horseid','追切指数']]

# カラムの整理
# 成績データファイルの列名を変更
rename_map = {
    'レースID(新)': 'target_raceid',
    'レースID(新).1': 'target_horseid',
    '上り3F': 'レース上り3F',
    '上り4F': 'レース上り4F',
    '上り5F': 'レース上り5F',
    '外部指数1':'レイティング',
    '外部指数順1':'レイティング順位',
    '外部指数2':'ZI指数',
    '外部指数順2':'ZI指数順位',
    '外部指数3':'追切指数',
    '外部指数順3':'追切指数順位',
    '上り3F.1':'上り3F'
}

# df1の列名に対してrename_mapを適用
merged_df1 = merged_df1.rename(columns=rename_map)

# キーの型を揃える（数字/ゼロ埋めブレ防止）
for c in ['target_horseid', 'target_raceid']:
    if c in merged_df1: merged_df1[c] = merged_df1[c].astype(str)
for d in [merged_df2, merged_df3, merged_df4, merged_df5, merged_df6, merged_df7, merged_df8, merged_df9]:
    for c in ['target_horseid', 'target_raceid']:
        if c in d: d[c] = d[c].astype(str)

#　種牡馬名のspace削除
merged_df1['種牡馬'] = merged_df1['種牡馬'].astype(str).str.replace(r'\s+', '', regex=True)

# ラベル化する列を追加
# コースラベル
insert_pos = merged_df1.columns.get_loc('コースグループ名1') + 1
merged_df1.insert(insert_pos, 'コースラベル', merged_df1['場所'].astype(str) + '_' + merged_df1['芝・ダート'].astype(str) + '_' + merged_df1['トラックコード(JV)'].astype(str))

# 父×母の父タイプ名
insert_pos = merged_df1.columns.get_loc('母の父タイプ名') + 1
merged_df1.insert(insert_pos, '父×母の父タイプ名', merged_df1['種牡馬'].astype(str) + '×' + merged_df1['母の父タイプ名'].astype(str))

# 父タイプ名×母の父タイプ名
insert_pos = merged_df1.columns.get_loc('父×母の父タイプ名') + 1
merged_df1.insert(insert_pos, '父タイプ名×母の父タイプ名', merged_df1['種牡馬タイプ名'].astype(str) + '×' + merged_df1['母の父タイプ名'].astype(str))

# 生産者×馬主
insert_pos = merged_df1.columns.get_loc('騎手') + 1
merged_df1.insert(insert_pos, '生産者×馬主', merged_df1['生産者'].astype(str) + '×' + merged_df1['馬主'].astype(str))

# 生産者×調教師
insert_pos = merged_df1.columns.get_loc('生産者×馬主') + 1
merged_df1.insert(insert_pos, '生産者×調教師', merged_df1['生産者'].astype(str) + '×' + merged_df1['調教師'].astype(str))

# 生産者×騎手
insert_pos = merged_df1.columns.get_loc('生産者×調教師') + 1
merged_df1.insert(insert_pos, '生産者×騎手', merged_df1['生産者'].astype(str) + '×' + merged_df1['騎手'].astype(str))

# 馬主×調教師
insert_pos = merged_df1.columns.get_loc('生産者×騎手') + 1
merged_df1.insert(insert_pos, '馬主×調教師', merged_df1['馬主'].astype(str) + '×' + merged_df1['調教師'].astype(str))

# 馬主×騎手
insert_pos = merged_df1.columns.get_loc('馬主×調教師') + 1
merged_df1.insert(insert_pos, '馬主×騎手', merged_df1['馬主'].astype(str) + '×' + merged_df1['騎手'].astype(str))

# 調教師×騎手
insert_pos = merged_df1.columns.get_loc('馬主×騎手') + 1
merged_df1.insert(insert_pos, '調教師×騎手', merged_df1['調教師'].astype(str) + '×' + merged_df1['騎手'].astype(str))

# 調教師×コースラベル
insert_pos = merged_df1.columns.get_loc('調教師×騎手') + 1
merged_df1.insert(insert_pos, '調教師×コースラベル', merged_df1['調教師'].astype(str) + '×' + merged_df1['コースラベル'].astype(str))

# 騎手×コースラベル
insert_pos = merged_df1.columns.get_loc('調教師×コースラベル') + 1
merged_df1.insert(insert_pos, '騎手×コースラベル', merged_df1['騎手'].astype(str) + '×' + merged_df1['コースラベル'].astype(str))

# 配当金額の列を作成する
# 列から取り出す金額の数を定義
target_cols = {
    '単勝配当表記': ('単勝', 1),
    '複勝配当表記': ('複勝', 3),
    '枠連配当表記': ('枠連', 1),
    '馬連配当表記': ('馬連', 1),
    'ワイド配当表記': ('ワイド', 3),
    '馬単配当表記': ('馬単', 1),
    '３連複配当表記': ('３連複', 1),
    '３連単配当表記': ('３連単', 1),
}

# 金額を取り出す正規表現パターン
pattern = re.compile(r'[\\¥]\s*([0-9,]+)(?=\s*(?:\(|/|$))')

def pick_amounts(text, take=1):
    if pd.isna(text):
        return [np.nan]*take
    s = str(text)
    found = pattern.findall(s)
    nums = []
    for x in found:
        try:
            nums.append(int(x.replace(',', '')))
        except:
            # 変な値が来てもスルー
            continue
    # 必要な個数だけ先頭から取り、足りなければNaNで埋める
    nums = nums[:take]
    if len(nums) < take:
        nums += [np.nan]*(take - len(nums))
    return nums

for src_col, (base_name, take) in target_cols.items():
    if src_col not in merged_df1.columns:
        # その列が無い場合はスキップ
        continue

    # 値を取り出す
    values = merged_df1[src_col].apply(lambda x: pick_amounts(x, take))

    # 1個だけなら「単勝」のように1列、3個なら「複勝1, 複勝2, 複勝3」の3列を作る
    if take == 1:
        col_name = base_name
        merged_df1[col_name] = values.apply(lambda v: v[0] if isinstance(v, list) else np.nan)
        # Convert to integer, coercing errors to NaN
        merged_df1[col_name] = pd.to_numeric(merged_df1[col_name], errors='coerce').astype('Int64')
    else:
        for i in range(take):
            col_name = f'{base_name}{i+1}'
            merged_df1[col_name] = values.apply(lambda v: v[i] if isinstance(v, list) and len(v) > i else np.nan)
            # Convert to integer, coercing errors to NaN
            merged_df1[col_name] = pd.to_numeric(merged_df1[col_name], errors='coerce').astype('Int64')

# 成績データファイルへ対象データをマージする
# 初角位置のマージ
merged_df1 = pd.merge(merged_df1, merged_df2, on='target_horseid', how='left')
# 2角位置のマージ
merged_df1 = pd.merge(merged_df1, merged_df3, on='target_horseid', how='left')
# 3角位置のマージ
merged_df1 = pd.merge(merged_df1, merged_df4, on='target_horseid', how='left')
# 4角位置のマージ
merged_df1 = pd.merge(merged_df1, merged_df5, on='target_horseid', how='left')
# 上り位置のマージ
merged_df1 = pd.merge(merged_df1, merged_df6, on='target_horseid', how='left')
# 馬場指数のマージ
merged_df1 = pd.merge(merged_df1, merged_df7, on='target_raceid', how='left')
# 前半3Fのマージ
merged_df1 = pd.merge(merged_df1, merged_df8, on='target_horseid', how='left')
# 追切指数の代入 とりあえずTrueで上書き
left  = merged_df1.set_index('target_horseid')
right = merged_df9[['target_horseid','追切指数']].dropna(subset=['追切指数']).set_index('target_horseid')
left.update(right, overwrite=True)
merged_df1 = left.reset_index()

# 不要なデータフレームをクリア
del merged_df2
del merged_df3
del merged_df4
del merged_df5
del merged_df6
del merged_df7
del merged_df8

# --- 関数群 ---
def calculate_33_lap(merged_df1):
    """距離ごとに33ラップを計算する"""
    rap_33 = []
    distance_to_lap_range = {
        1000: (0, 3),
        1200: (0, 3),
        1300: (1, 4),
        1400: (1, 4),
        1500: (2, 5),
        1600: (2, 5),
        1700: (3, 6),
        1800: (3, 6),
        1900: (4, 7),
        2000: (4, 7),
        2100: (5, 8),
        2200: (5, 8),
        2300: (6, 9),
        2400: (6, 9),
        2500: (7, 10),
        2600: (7, 10),
        2700: (8, 11),
        2800: (8, 11),
        2900: (9, 12),
        3000: (9, 12),
        3100: (10, 13),
        3200: (10, 13),
        3300: (11, 14),
        3400: (11, 14),
        3500: (12, 15),
        3600: (12, 15),
    }
    for _, row in merged_df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_lap_range:
            start, end = distance_to_lap_range[distance]
            sum_6to4 = sum(lap_times[start:end])
        elif distance == 1150:
            sum_6to4 = round(lap_times[0] * 1.25, 1) + lap_times[1] + lap_times[2] if len(lap_times) >= 3 else None
        else:
            sum_6to4 = None
        sum_3to1 = row['レース上り3F'] if 'レース上り3F' in row and not pd.isna(row['レース上り3F']) else None
        rap_33_value = sum_6to4 - sum_3to1 if not pd.isna(sum_6to4) and not pd.isna(sum_3to1) else None
        rap_33.append(rap_33_value)
    merged_df1['33ラップ'] = rap_33
    return merged_df1

def calculate_middle_lap(merged_df1):
    """距離ごとに中盤ラップ1・2を計算する"""
    middle_lap1 = []
    middle_lap2 = []
    distance_to_mid_lap = {
        1000: (2, 4, None, None),
        1150: (2, 4, None, None),
        1200: (2, 4, None, None),
        1300: (2, 4, None, None),
        1400: (2, 4, None, None),
        1500: (3, 5, None, None),
        1600: (3, 5, None, None),
        1700: (3, 6, None, None),
        1800: (3, 6, None, None),
        1900: (3, 7, None, None),
        2000: (3, 7, None, None),
        2100: (3, 5, 5, 8),
        2200: (3, 5, 5, 8),
        2300: (3, 5, 5, 8),
        2400: (3, 6, 6, 9),
        2500: (3, 6, 6, 9),
        2600: (3, 7, 7, 10),
        3000: (3, 8, 8, 12),
        3200: (3, 8, 8, 13),
        3400: (3, 9, 9, 14),
        3600: (3, 9, 9, 15),
    }
    for _, row in merged_df1.iterrows():
        distance = row['距離']
        lap_times = [row[f'Lap{str(i+1).zfill(2)}'] for i in range(25)]
        lap_times = [t for t in lap_times if not pd.isna(t)]
        if distance in distance_to_mid_lap:
            mid1_start, mid1_end, mid2_start, mid2_end = distance_to_mid_lap[distance]
            mid1 = sum(lap_times[mid1_start:mid1_end]) if mid1_start is not None else None
            mid2 = sum(lap_times[mid2_start:mid2_end]) if mid2_start is not None else None
        else:
            mid1 = None
            mid2 = None
        middle_lap1.append(mid1)
        middle_lap2.append(mid2)
    merged_df1['中盤ラップ1'] = middle_lap1
    merged_df1['中盤ラップ2'] = middle_lap2
    return merged_df1

def calculate_lap_features(df):
    """
    Lap01～Lap25 を使って
      ・最大加速（隣接ラップ差分の最小値）
      ・ゴール前ラップ差（ラスト1F - ラスト2F）
    を計算して df に列を追加する。
    """
    lap_cols = [f'Lap{str(i).zfill(2)}' for i in range(1, 26)]

    def _calc_row(row):
        # その馬のラップ一覧（NaN は除外）
        laps = []
        for c in lap_cols:
            if c in row.index and pd.notna(row[c]):
                laps.append(row[c])

        # ラップが1つ以下ならどっちも計算不能
        if len(laps) < 2:
            return pd.Series({'最大加速ラップ': np.nan, 'ゴール前ラップ差': np.nan})

        laps = np.array(laps, dtype=float)

        # 隣り合う差分（後ろ - 前）
        diffs = np.diff(laps)   # 例：Lap02-Lap01, Lap03-Lap02, ...

        # 最大加速 = 最もマイナスが大きい差分（＝最小値）
        max_accel = diffs.min() if len(diffs) > 0 else np.nan

        # 終盤ラップ差 = ラスト1F - ラスト2F
        last_diff = laps[-1] - laps[-2] if len(laps) >= 2 else np.nan

        return pd.Series({
            '最大加速ラップ': max_accel if pd.notna(max_accel) else np.nan,
            'ゴール前ラップ差': last_diff if pd.notna(last_diff) else np.nan
        })

    new_cols = df.apply(_calc_row, axis=1)
    df['最大加速ラップ'] = new_cols['最大加速ラップ'].round(1)
    df['ゴール前ラップ差'] = new_cols['ゴール前ラップ差'].round(1)

    return df

def calculate_days_since_birth(merged_df1):
    """生後日数を計算する"""
    birth_days = []
    for _, row in merged_df1.iterrows():
        race_date = datetime.strptime(row['日付S'], '%Y.%m.%d')
        birth_str = row['誕生日'].replace(" ", "").replace("日", "").replace("-", "")
        birth_month, birth_day = map(int, birth_str.replace("月", " ").split())
        birth_year = race_date.year - row['年齢']
        try:
            birth_date = datetime(birth_year, birth_month, birth_day)
        except ValueError:
            birth_date = datetime(birth_year, 2, 28)
        days_old = (race_date - birth_date).days
        birth_days.append(days_old)
    merged_df1['生後日数'] = birth_days
    return merged_df1

def calculate_distance_diff(merged_df1):
    """前走距離との差を計算する"""
    merged_df1['前走距離差'] = merged_df1['距離'] - merged_df1['前走距離']
    return merged_df1

def calculate_firsthalf_diff(merged_df1):
    """初角から4角位置の差分を計算"""
    merged_df1['初角_4角差'] = merged_df1.apply(lambda row:
        (row['通過1'] - row['通過4']) if pd.notna(row['通過1']) else \
        ((row['通過2'] - row['通過4']) if pd.notna(row['通過2']) else \
        ((row['通過3'] - row['通過4']) if pd.notna(row['通過3']) else np.nan))
    , axis=1).fillna(0)
    return merged_df1

def calculate_goal_diff(merged_df1):
    """4角から入線順位の差分を計算"""
    # 入線順位を一時的に数値化
    rank_num = pd.to_numeric(merged_df1['入線順位'], errors='coerce')

    # 有効な順位（1以上）だけを判定するためのマスク
    valid_mask = rank_num >= 1

    # 出力列だけ作る
    merged_df1['4角_入線順位差'] = np.nan

    # 有効な行だけ計算
    merged_df1.loc[valid_mask, '4角_入線順位差'] = (
        pd.to_numeric(merged_df1.loc[valid_mask, '通過4'], errors='coerce')
        - rank_num[valid_mask]
    )

    return merged_df1

def calculate_sideposition(merged_df1):
    """サイドポジションの平均を計算する"""
    cols = ['初角サイドポジション', '2角サイドポジション', '3角サイドポジション', '4角サイドポジション']
    merged_df1['サイドポジション平均'] = merged_df1[cols].mean(axis=1)
    return merged_df1

def calculate_totalprize(merged_df1):
    """獲得賞金を計算する"""
    merged_df1['獲得賞金'] = merged_df1['賞金'].fillna(0) + merged_df1['付加賞金'].fillna(0)
    return merged_df1

def convert_time_to_seconds(time_str):
    """タイム表記を秒数に変換"""
    try:
        parts = time_str.split(".")
        if len(parts) == 3:
            minutes, seconds, tenths = map(int, parts)
            total_seconds = minutes * 60 + seconds + tenths * 0.1
        else:
            return np.nan
        return total_seconds
    except:
        return np.nan

def remove_plus_sign(value):
    """数値データから `+` を削除して変換"""
    try:
        return float(str(value).replace("+", ""))
    except:
        return np.nan

def extract_weight(value):
    """斤量の数値部分だけを抽出"""
    try:
        match = re.search(r'\d+', str(value))
        return int(match.group()) if match else np.nan
    except:
        return np.nan

def calculate_corner_loss(row):
    """コーナーロスを計算"""
    corner_positions = [
        row.get('初角サイドポジション', 1) - 1,
        row.get('2角サイドポジション', 1) - 1,
        row.get('3角サイドポジション', 1) - 1,
        row.get('4角サイドポジション', 1) - 1
    ]
    total_distance_loss = sum(corner_positions) * 1.5  # m単位
    finish_time_seconds = row['タイムS']
    distance_m = row['距離']
    if pd.isna(finish_time_seconds) or pd.isna(distance_m) or finish_time_seconds == 0:
        return np.nan
    avg_speed = distance_m / finish_time_seconds if finish_time_seconds > 0 else np.nan
    corner_loss = total_distance_loss / avg_speed if avg_speed > 0 else 0
    return round(corner_loss, 2)

# 各列の整形
# 対象の列を数値変換
merged_df1['馬場指数'] = merged_df1['馬場指数'].astype(str).str.extract(r'(-?\d+)')[0].astype('Int64')
merged_df1['レイティング'] = pd.to_numeric(merged_df1['レイティング'], errors='coerce')
merged_df1['体重'] = pd.to_numeric(merged_df1['体重'], errors='coerce')
merged_df1['Ave-3F'] = pd.to_numeric(merged_df1['Ave-3F'], errors='coerce')
merged_df1['上り3F'] = pd.to_numeric(merged_df1['上り3F'], errors='coerce')

# 対象の列から記号を除去
for col in ['前後3F差', '前後4F差', '前後5F差', '増減']:
    merged_df1[col] = merged_df1[col].apply(remove_plus_sign)

merged_df1['斤量'] = merged_df1['斤量'].apply(extract_weight)

# タイムを秒数に変換
merged_df1['タイムS'] = merged_df1['タイムS'].apply(convert_time_to_seconds)
merged_df1['-3Fタイム'] = merged_df1['-3Fタイム'].apply(convert_time_to_seconds)

# 決め手列を変換(マップにない場合はNanにする)
lq_mapping = {
    '中団': '差し',
    '後方': '追込',
}

s = merged_df1['決め手'].astype('string').str.strip()
merged_df1['決め手'] = s.map(lq_mapping)

# 各種計算関数を順次実行
# 33ラップの計算
merged_df1 = calculate_33_lap(merged_df1)
# 中盤ラップ1・2の計算
merged_df1 = calculate_middle_lap(merged_df1)
# 最大加速ラップ・ゴール前ラップ差の計算
merged_df1 = calculate_lap_features(merged_df1)
# 生後日数の計算
merged_df1 = calculate_days_since_birth(merged_df1)
# 前走距離差の計算
merged_df1 = calculate_distance_diff(merged_df1)
# 初角から4角通過順位差の計算
merged_df1 = calculate_firsthalf_diff(merged_df1)
# 4角から入線順位差の計算
merged_df1 = calculate_goal_diff(merged_df1)
# サイドポジション平均値の計算
merged_df1 = calculate_sideposition(merged_df1)
# 獲得賞金の計算
merged_df1 = calculate_totalprize(merged_df1)
# 基準斤量の計算
merged_df1['基準斤量'] = merged_df1['斤量'] - merged_df1['馬齢斤量差']
# RPCI差の計算
merged_df1['RPCI差'] = merged_df1['PCI'] - merged_df1['レースPCI']
# コーナーロスの計算
merged_df1['コーナーロス'] = merged_df1.apply(calculate_corner_loss, axis=1)
# 補正走破タイムの計算
merged_df1['補正走破タイム'] = merged_df1['タイムS'] - merged_df1['コーナーロス']
# スローorハイ関数の計算
merged_df1['スローorハイ関数'] = merged_df1['Ave-3F'] - merged_df1['上り3F']

# マージ用の列作成処理
def categorize_race_type(race_type):
    if race_type == 11:
        return 'サラブレッド系2歳'
    elif race_type == 12:
        return 'サラブレッド系3歳'
    elif race_type >= 13:
        return 'サラブレッド系3歳以上'
    else:
        return np.nan

def categorize_class_code(class_code):
    if 7 <= class_code <= 15:
        return '新馬・未勝利'
    elif class_code == 23:
        return '1勝クラス'
    elif class_code == 43:
        return '2勝クラス'
    elif class_code == 67:
        return '3勝クラス'
    elif class_code >= 115:
        return 'OP・重賞'
    else:
        return np.nan

# 範囲定義
ranges = [ (-np.inf, -4.6), (-4.6, -3.6), (-3.6, -2.6), (-2.6, -1.6), (-1.6, -0.6), (-0.6, 0.6), (0.6, 1.6), (1.6, 2.6), (2.6, 3.6), (3.6, 4.6), (4.6, np.inf) ]

# 区間ラベル
def assign_range(value):
    if pd.isna(value):
        return np.nan
    for i, (lower, upper) in enumerate(ranges):
        if i < len(ranges) - 1:
            if lower <= value < upper:
                return f"{lower}～{upper}"
        else:
            # 最終区間（4.6～inf）は右も含める
            if lower <= value <= upper:
                return f"{lower}～{upper}"
    return np.nan  # 念のため

# 馬場分類・競走種別・クラス分類・スローor関数範囲の列を作成する
merged_df1['馬場分類'] = merged_df1['馬場状態'].replace({'良': '良・稍重','稍': '良・稍重','重': '重・不良','不': '重・不良'})
merged_df1['競走種別'] = merged_df1['年齢限定(競走種別コード)'].apply(categorize_race_type)
merged_df1['クラス分類'] = merged_df1['クラスコード'].apply(categorize_class_code)
merged_df1['スローorハイ関数範囲'] = merged_df1['スローorハイ関数'].apply(assign_range)

# 共通関数：参照側DFのキー以外の列にサフィックスを付ける
def prepare_ref_df(df_ref, merge_keys, suffix):
    """
    マージ用の参照DFの列名に suffix を付ける（キー列以外すべて）
    """
    df_ref = df_ref.copy()
    rename_map = {
        c: f"{c}{suffix}"
        for c in df_ref.columns
        if c not in merge_keys
    }
    df_ref = df_ref.rename(columns=rename_map)
    return df_ref

# マージキー
merge_key1 = ['場所','芝・ダート','距離','トラックコード(JV)','馬場分類']
merge_key2 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類','馬場分類']
merge_key3 = ['場所','芝・ダート','距離','トラックコード(JV)','スローorハイ関数範囲']
merge_key4 = ['場所','芝・ダート','距離','トラックコード(JV)']
merge_key5 = ['場所','芝・ダート','距離','トラックコード(JV)','競走種別','クラス分類']

# 基準タイム等データフレームの読み込み
base_time_df1 = pd.read_csv(ref_filepath1, encoding='cp932')
base_time_df2 = pd.read_csv(ref_filepath2, encoding='cp932')
pace_medians_df = pd.read_csv(ref_filepath3, encoding='cp932')
lap33_df = pd.read_csv(ref_filepath4, encoding='cp932')
racelevel_df = pd.read_csv(ref_filepath5, encoding='cp932')

# 参照側の列名にsuffixを付与
base_time_df1 = prepare_ref_df(base_time_df1, merge_key1, '_stdtime1').copy()
base_time_df2 = prepare_ref_df(base_time_df2, merge_key2, '_stdtime2').copy()
pace_medians_df = prepare_ref_df(pace_medians_df, merge_key3, '_paceindex').copy()
lap33_df = prepare_ref_df(lap33_df, merge_key4, '_33lap').copy()
racelevel_df = prepare_ref_df(racelevel_df, merge_key5, '_racelevel').copy()

# merged_df1へその他データフレームをマージ
merged_df1 = pd.merge(merged_df1, base_time_df1, on=merge_key1, how='left')
merged_df1 = pd.merge(merged_df1, base_time_df2, on=merge_key2, how='left')
merged_df1 = pd.merge(merged_df1, pace_medians_df, on=merge_key3, how='left')
merged_df1 = pd.merge(merged_df1, lap33_df, on=merge_key4, how='left')
merged_df1 = pd.merge(merged_df1, racelevel_df, on=merge_key5, how='left')

# ---ステップ３：成績データに指数追加
# 各指数算出の前処理
# 基準タイム/距離係数のベース列を作る（Std1優先、なければStd2）
base_time = merged_df1['タイムS_stdtime1'].fillna(merged_df1['タイムS_stdtime2'])
dist_coef = merged_df1['距離係数_stdtime1'].fillna(merged_df1['距離係数_stdtime2'])

# 基準タイム差（秒）：基準タイム - 補正走破タイム
base_time_diff = base_time - merged_df1['補正走破タイム']

# 基準タイム差×距離係数
speed_core = base_time_diff * dist_coef

# 平均3F補正値
ave3f_base = merged_df1['Ave-3F_stdtime1'].fillna(merged_df1['Ave-3F_stdtime2'])
revi_Ave3F = ave3f_base - merged_df1['Ave-3F']

# 斤量補正値
revi_weight = merged_df1['斤量'] - merged_df1['基準斤量']

# クラス補正値（Std1が存在する行だけ有効。Std2代用行では0）
# ※ Std1が無い = その条件の1・2勝基準が作れない想定なので、クラス補正は入れない
revi_class = (merged_df1['タイムS_stdtime1'] - merged_df1['タイムS_stdtime2']).where(
    merged_df1['タイムS_stdtime1'].notna(), 0
)

# ペース補正値
revi_pace = merged_df1['スローorハイ関数'] * merged_df1['ペース補正係数_paceindex']

# 馬場指数秒数換算
merged_df1['馬場補正値'] = merged_df1['馬場指数'] / 10

# 初角位置
merged_df1['初角位置'] = (
    merged_df1['通過1']
      .fillna(merged_df1['通過2'])
      .fillna(merged_df1['通過3'])
      .fillna(merged_df1['通過4'])
)

# テン指数の計算
merged_df1['テン指数'] = round((
(merged_df1['前半3F_stdtime1'].fillna(merged_df1['前半3F_stdtime2']) - merged_df1['前半3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数'] - merged_df1['初角位置'] + 1) / merged_df1['頭数']) # 初角の位置取り評価
+ 50
), 1)

# 上り指数の計算
merged_df1['上り指数'] = round((
(merged_df1['上り3F_stdtime1'].fillna(merged_df1['上り3F_stdtime2']) - merged_df1['上り3F']) # 基準タイムとの比較評価
+ ((merged_df1['頭数']) - merged_df1['入線順位'] + 1) / merged_df1['頭数'] # 着順評価
+ (merged_df1['通過4'] - merged_df1['入線順位']) / merged_df1['頭数'] # ポジション押し上げ力評価
+ (merged_df1['頭数'] - merged_df1['通過4'] + 1) / merged_df1['頭数'] # 4角の位置取り評価
+ 50
), 1)

# スピード指数の計算 ---
merged_df1['スピード指数'] = (
    (
        speed_core
        + merged_df1['馬場補正値'].fillna(0)
        + revi_Ave3F.fillna(0)
        + revi_weight.fillna(0)
        + revi_class.fillna(0)
        + revi_pace.fillna(0)
        + 100
    )
    .round(1)
    .where(speed_core.notna() & (speed_core != 0))
)

# 総合指数の計算
merged_df1['総合指数'] = merged_df1[['スピード指数', '補正タイム', '補9']].mean(axis=1).round(1)

# 総合指数の代表値(1着～3着)をdf1へ追加
central_score_df1 = merged_df1[merged_df1['入線順位'].between(1, 3)].copy()
central_score_df1 = central_score_df1.groupby('target_raceid' ,as_index = False)['総合指数'].mean()
central_score_df1.rename(columns={'総合指数': 'Top3総合指数'}, inplace=True)
central_score_df1['Top3総合指数'] = round(central_score_df1['Top3総合指数'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df1, on='target_raceid', how='left',suffixes=('', '_centralscore1')).copy()

# レイティングの平均値を成績データへ追加
merged_df1['レイティング'] = pd.to_numeric(merged_df1['レイティング'], errors='coerce')
central_score_df2 = merged_df1.groupby('target_raceid' ,as_index = False)['レイティング'].mean().copy()
central_score_df2.rename(columns={'レイティング': 'レイティング平均値'}, inplace=True)
central_score_df2['レイティング平均値'] = round(central_score_df2['レイティング平均値'], 1)
merged_df1 = pd.merge(merged_df1, central_score_df2, on='target_raceid', how='left' ,suffixes=('', '_centralscore2')).copy()

# 不要なデータフレームのクリア
del central_score_df1
del central_score_df2

# 各指数の偏差値を計算
def deviation_in_race(series):
    mean = series.mean()
    std = series.std()
    if pd.isna(std) or std == 0:
        return pd.Series(np.nan, index=series.index)
    return ((series - mean) / std * 10 + 50)

# レース内偏差値
merged_df1['レイティング偏差値'] = (merged_df1.groupby('target_raceid')['レイティング'].transform(deviation_in_race).round(1))
merged_df1['ZI指数偏差値'] = (merged_df1.groupby('target_raceid')['ZI指数'].transform(deviation_in_race).round(1))
merged_df1['追切指数偏差値'] = (merged_df1.groupby('target_raceid')['追切指数'].transform(deviation_in_race).round(1))
merged_df1['マイニング偏差値'] = (merged_df1.groupby('target_raceid')['マイニング'].transform(deviation_in_race).round(1))
merged_df1['対戦型マイニング偏差値'] = (merged_df1.groupby('target_raceid')['対戦型マイニング'].transform(deviation_in_race).round(1))
merged_df1['前半3F偏差値'] = merged_df1.groupby('target_raceid')['前半3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['上り3F偏差値'] = merged_df1.groupby('target_raceid')['上り3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['Ave-3F偏差値'] = merged_df1.groupby('target_raceid')['Ave-3F'].transform(lambda s: deviation_in_race(-s)).round(1)
merged_df1['テン指数偏差値'] = (merged_df1.groupby('target_raceid')['テン指数'].transform(deviation_in_race).round(1))
merged_df1['上り指数偏差値'] = (merged_df1.groupby('target_raceid')['上り指数'].transform(deviation_in_race).round(1))
merged_df1['スピード指数偏差値'] = (merged_df1.groupby('target_raceid')['スピード指数'].transform(deviation_in_race).round(1))
merged_df1['総合指数偏差値'] = (merged_df1.groupby('target_raceid')['総合指数'].transform(deviation_in_race).round(1))

# レース内順位（dense）
merged_df1['前半3F順位'] = (merged_df1.groupby('target_raceid')['前半3F'].rank(ascending=True, method='dense').astype('Int64'))
merged_df1['テン指数順位'] = (merged_df1.groupby('target_raceid')['テン指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['上り指数順位'] = (merged_df1.groupby('target_raceid')['上り指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['Ave-3F順位'] = (merged_df1.groupby('target_raceid')['Ave-3F'].rank(ascending=True, method='dense').astype('Int64'))
merged_df1['スピード指数順位'] = (merged_df1.groupby('target_raceid')['スピード指数'].rank(ascending=False, method='dense').astype('Int64'))
merged_df1['総合指数順位'] = (merged_df1.groupby('target_raceid')['総合指数'].rank(ascending=False, method='dense').astype('Int64'))

# 順位分布の計算
merged_df1['レイティング順位分布'] = (merged_df1['レイティング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['ZI指数順位分布'] = (merged_df1['ZI指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['追切指数順位分布'] = (merged_df1['追切指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['マイニング順位分布'] = (merged_df1['マイニング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['対戦型マイニング順位分布'] = (merged_df1['対戦型マイニング順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['前半3F順位分布'] = (merged_df1['前半3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['テン指数順位分布'] = (merged_df1['テン指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['上り3F順位分布'] = (merged_df1['上り3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['上り指数順位分布'] = (merged_df1['上り指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['Ave-3F順位分布'] = (merged_df1['Ave-3F順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['スピード指数順位分布'] = (merged_df1['スピード指数順位'] - 1) / (merged_df1['頭数'] - 1)
merged_df1['総合指数順位分布'] = (merged_df1['総合指数順位'] - 1) / (merged_df1['頭数'] - 1)

# ---ステップ４：基準タイムとの指数差追加
# Top3総合指数と総合指数の差分を成績データへ追加
merged_df1['Top3総合指数差分'] = merged_df1['総合指数'] - merged_df1['Top3総合指数']

# レイティング平均値とレイティングの差分を追加
merged_df1['平均レイティング差分'] = merged_df1['レイティング'] - merged_df1['レイティング平均値']

# PCI3：基準タイム１差分
merged_df1['PCI3差分']=merged_df1['PCI3']-merged_df1['PCI3_stdtime1'].fillna(merged_df1['PCI3_stdtime2'])

# PCI：基準タイム１差分
merged_df1['PCI上位差分']=merged_df1['PCI']-merged_df1['PCI_stdtime1'].fillna(merged_df1['PCI_stdtime2'])
merged_df1['PCI勝馬差分']=merged_df1['PCI']-merged_df1['PCI_1着_stdtime1'].fillna(merged_df1['PCI_1着_stdtime2'])

# 生後日数：基準タイム２差分
merged_df1['生後日数上位差分']=merged_df1['生後日数']-merged_df1['生後日数_stdtime2'].fillna(merged_df1['生後日数_stdtime1'])
merged_df1['生後日数勝馬差分']=merged_df1['生後日数']-merged_df1['生後日数_1着_stdtime2'].fillna(merged_df1['生後日数_1着_stdtime1'])

# 馬体重：基準タイム２差分
merged_df1['体重上位差分']=merged_df1['体重']-merged_df1['体重_stdtime2'].fillna(merged_df1['体重_stdtime1'])
merged_df1['体重勝馬差分']=merged_df1['体重']-merged_df1['体重_1着_stdtime2'].fillna(merged_df1['体重_1着_stdtime1'])

# 馬体重斤量比：基準タイム２差分
merged_df1['斤量馬体重比上位差分']=merged_df1['斤量馬体重比']-merged_df1['斤量馬体重比_stdtime2'].fillna(merged_df1['斤量馬体重比_stdtime1'])
merged_df1['斤量馬体重比勝馬差分']=merged_df1['斤量馬体重比']-merged_df1['斤量馬体重比_1着_stdtime2'].fillna(merged_df1['斤量馬体重比_1着_stdtime1'])

# 前走距離：基準タイム１差分
merged_df1['前走距離上位差分']=merged_df1['前走距離']-merged_df1['前走距離_stdtime1'].fillna(merged_df1['前走距離_stdtime2'])
merged_df1['前走距離勝馬差分']=merged_df1['前走距離']-merged_df1['前走距離_1着_stdtime1'].fillna(merged_df1['前走距離_1着_stdtime2'])

# 前走距離差：基準タイム１差分
merged_df1['前走距離差上位差分']=merged_df1['前走距離差']-merged_df1['前走距離差_stdtime1'].fillna(merged_df1['前走距離差_stdtime2'])
merged_df1['前走距離差勝馬差分']=merged_df1['前走距離差']-merged_df1['前走距離差_1着_stdtime1'].fillna(merged_df1['前走距離差_1着_stdtime2'])

# レイティング：基準タイム１差分
merged_df1['レイティング上位差分']=merged_df1['レイティング']-merged_df1['レイティング_stdtime1'].fillna(merged_df1['レイティング_stdtime2'])
merged_df1['レイティング勝馬差分']=merged_df1['レイティング']-merged_df1['レイティング_1着_stdtime1'].fillna(merged_df1['レイティング_1着_stdtime2'])

# レイティング偏差値：基準タイム１差分
merged_df1['レイティング偏差値上位差分']=merged_df1['レイティング偏差値']-merged_df1['レイティング偏差値_stdtime1'].fillna(merged_df1['レイティング偏差値_stdtime2'])
merged_df1['レイティング偏差値勝馬差分']=merged_df1['レイティング偏差値']-merged_df1['レイティング偏差値_1着_stdtime1'].fillna(merged_df1['レイティング偏差値_1着_stdtime2'])

# レイティング順位分布：基準タイム１差分
merged_df1['レイティング順位分布上位差分']=merged_df1['レイティング順位分布']-merged_df1['レイティング順位分布_stdtime1'].fillna(merged_df1['レイティング順位分布_stdtime2'])
merged_df1['レイティング順位分布勝馬差分']=merged_df1['レイティング順位分布']-merged_df1['レイティング順位分布_1着_stdtime1'].fillna(merged_df1['レイティング順位分布_1着_stdtime2'])

# ZI指数：基準タイム１差分
merged_df1['ZI指数上位差分']=merged_df1['ZI指数']-merged_df1['ZI指数_stdtime1'].fillna(merged_df1['ZI指数_stdtime2'])
merged_df1['ZI指数勝馬差分']=merged_df1['ZI指数']-merged_df1['ZI指数_1着_stdtime1'].fillna(merged_df1['ZI指数_1着_stdtime2'])

# ZI指数偏差値：基準タイム１差分
merged_df1['ZI指数偏差値上位差分']=merged_df1['ZI指数偏差値']-merged_df1['ZI指数偏差値_stdtime1'].fillna(merged_df1['ZI指数偏差値_stdtime2'])
merged_df1['ZI指数偏差値勝馬差分']=merged_df1['ZI指数偏差値']-merged_df1['ZI指数偏差値_1着_stdtime1'].fillna(merged_df1['ZI指数偏差値_1着_stdtime2'])

# ZI指数順位分布：基準タイム１差分
merged_df1['ZI指数順位分布上位差分']=merged_df1['ZI指数順位分布']-merged_df1['ZI指数順位分布_stdtime1'].fillna(merged_df1['ZI指数順位分布_stdtime2'])
merged_df1['ZI指数順位分布勝馬差分']=merged_df1['ZI指数順位分布']-merged_df1['ZI指数順位分布_1着_stdtime1'].fillna(merged_df1['ZI指数順位分布_1着_stdtime2'])

# 追切指数：基準タイム１差分
merged_df1['追切指数上位差分']=merged_df1['追切指数']-merged_df1['追切指数_stdtime1'].fillna(merged_df1['追切指数_stdtime2'])
merged_df1['追切指数勝馬差分']=merged_df1['追切指数']-merged_df1['追切指数_1着_stdtime1'].fillna(merged_df1['追切指数_1着_stdtime2'])

# 追切指数偏差値：基準タイム１差分
merged_df1['追切指数偏差値上位差分']=merged_df1['追切指数偏差値']-merged_df1['追切指数偏差値_stdtime1'].fillna(merged_df1['追切指数偏差値_stdtime2'])
merged_df1['追切指数偏差値勝馬差分']=merged_df1['追切指数偏差値']-merged_df1['追切指数偏差値_1着_stdtime1'].fillna(merged_df1['追切指数偏差値_1着_stdtime2'])

# 追切指数順位分布：基準タイム１差分
merged_df1['追切指数順位分布上位差分']=merged_df1['追切指数順位分布']-merged_df1['追切指数順位分布_stdtime1'].fillna(merged_df1['追切指数順位分布_stdtime2'])
merged_df1['追切指数順位分布勝馬差分']=merged_df1['追切指数順位分布']-merged_df1['追切指数順位分布_1着_stdtime1'].fillna(merged_df1['追切指数順位分布_1着_stdtime2'])

# マイニング：基準タイム１差分
merged_df1['マイニング上位差分']=merged_df1['マイニング']-merged_df1['マイニング_stdtime1'].fillna(merged_df1['マイニング_stdtime2'])
merged_df1['マイニング勝馬差分']=merged_df1['マイニング']-merged_df1['マイニング_1着_stdtime1'].fillna(merged_df1['マイニング_1着_stdtime2'])

# マイニング偏差値：基準タイム１差分
merged_df1['マイニング偏差値上位差分']=merged_df1['マイニング偏差値']-merged_df1['マイニング偏差値_stdtime1'].fillna(merged_df1['マイニング偏差値_stdtime2'])
merged_df1['マイニング偏差値勝馬差分']=merged_df1['マイニング偏差値']-merged_df1['マイニング偏差値_1着_stdtime1'].fillna(merged_df1['マイニング偏差値_1着_stdtime2'])

# マイニング順位分布：基準タイム１差分
merged_df1['マイニング順位分布上位差分']=merged_df1['マイニング順位分布']-merged_df1['マイニング順位分布_stdtime1'].fillna(merged_df1['マイニング順位分布_stdtime2'])
merged_df1['マイニング順位分布勝馬差分']=merged_df1['マイニング順位分布']-merged_df1['マイニング順位分布_1着_stdtime1'].fillna(merged_df1['マイニング順位分布_1着_stdtime2'])

# 対戦型マイニング：基準タイム１差分
merged_df1['対戦型マイニング上位差分']=merged_df1['対戦型マイニング']-merged_df1['対戦型マイニング_stdtime1'].fillna(merged_df1['対戦型マイニング_stdtime2'])
merged_df1['対戦型マイニング勝馬差分']=merged_df1['対戦型マイニング']-merged_df1['対戦型マイニング_1着_stdtime1'].fillna(merged_df1['対戦型マイニング_1着_stdtime2'])

# 対戦型マイニング偏差値：基準タイム１差分
merged_df1['対戦型マイニング偏差値上位差分']=merged_df1['対戦型マイニング偏差値']-merged_df1['対戦型マイニング偏差値_stdtime1'].fillna(merged_df1['対戦型マイニング偏差値_stdtime2'])
merged_df1['対戦型マイニング偏差値勝馬差分']=merged_df1['対戦型マイニング偏差値']-merged_df1['対戦型マイニング偏差値_1着_stdtime1'].fillna(merged_df1['対戦型マイニング偏差値_1着_stdtime2'])

# 対戦型マイニング順位分布：基準タイム１差分
merged_df1['対戦型マイニング順位分布上位差分']=merged_df1['対戦型マイニング順位分布']-merged_df1['対戦型マイニング順位分布_stdtime1'].fillna(merged_df1['対戦型マイニング順位分布_stdtime2'])
merged_df1['対戦型マイニング順位分布勝馬差分']=merged_df1['対戦型マイニング順位分布']-merged_df1['対戦型マイニング順位分布_1着_stdtime1'].fillna(merged_df1['対戦型マイニング順位分布_1着_stdtime2'])

# タイムS：基準タイム１差分
merged_df1['タイムS上位差分']=merged_df1['タイムS']-merged_df1['タイムS_stdtime1'].fillna(merged_df1['タイムS_stdtime2'])
merged_df1['タイムS勝馬差分']=merged_df1['タイムS']-merged_df1['タイムS_1着_stdtime1'].fillna(merged_df1['タイムS_1着_stdtime2'])

# 補正走破タイム：基準タイム１差分
merged_df1['補正走破タイム上位差分']=merged_df1['補正走破タイム']-merged_df1['補正走破タイム_stdtime1'].fillna(merged_df1['補正走破タイム_stdtime2'])
merged_df1['補正走破タイム勝馬差分']=merged_df1['補正走破タイム']-merged_df1['補正走破タイム_1着_stdtime1'].fillna(merged_df1['補正走破タイム_1着_stdtime2'])

# -3Fタイム：基準タイム１差分
merged_df1['-3Fタイム上位差分']=merged_df1['-3Fタイム']-merged_df1['-3Fタイム_stdtime1'].fillna(merged_df1['-3Fタイム_stdtime2'])
merged_df1['-3Fタイム勝馬差分']=merged_df1['-3Fタイム']-merged_df1['-3Fタイム_1着_stdtime1'].fillna(merged_df1['-3Fタイム_1着_stdtime2'])

# 前半3F：基準タイム１差分
merged_df1['前半3F上位差分']=merged_df1['前半3F']-merged_df1['前半3F_stdtime1'].fillna(merged_df1['前半3F_stdtime2'])
merged_df1['前半3F勝馬差分']=merged_df1['前半3F']-merged_df1['前半3F_1着_stdtime1'].fillna(merged_df1['前半3F_1着_stdtime2'])

# 前半3F偏差値：基準タイム１差分
merged_df1['前半3F偏差値上位差分']=merged_df1['前半3F偏差値']-merged_df1['前半3F偏差値_stdtime1'].fillna(merged_df1['前半3F偏差値_stdtime2'])
merged_df1['前半3F偏差値勝馬差分']=merged_df1['前半3F偏差値']-merged_df1['前半3F偏差値_1着_stdtime1'].fillna(merged_df1['前半3F偏差値_1着_stdtime2'])

# 前半3F順位：基準タイム１差分
merged_df1['前半3F順位上位差分']=merged_df1['前半3F順位']-merged_df1['前半3F順位_stdtime1'].fillna(merged_df1['前半3F順位_stdtime2'])
merged_df1['前半3F順位勝馬差分']=merged_df1['前半3F順位']-merged_df1['前半3F順位_1着_stdtime1'].fillna(merged_df1['前半3F順位_1着_stdtime2'])

# 前半3F順位分布：基準タイム１差分
merged_df1['前半3F順位分布上位差分']=merged_df1['前半3F順位分布']-merged_df1['前半3F順位分布_stdtime1'].fillna(merged_df1['前半3F順位分布_stdtime2'])
merged_df1['前半3F順位分布勝馬差分']=merged_df1['前半3F順位分布']-merged_df1['前半3F順位分布_1着_stdtime1'].fillna(merged_df1['前半3F順位分布_1着_stdtime2'])

# 上り3F：基準タイム１差分
merged_df1['上り3F上位差分']=merged_df1['上り3F']-merged_df1['上り3F_stdtime1'].fillna(merged_df1['上り3F_stdtime2'])
merged_df1['上り3F勝馬差分']=merged_df1['上り3F']-merged_df1['上り3F_1着_stdtime1'].fillna(merged_df1['上り3F_1着_stdtime2'])

# 上り3F偏差値：基準タイム１差分
merged_df1['上り3F偏差値上位差分']=merged_df1['上り3F偏差値']-merged_df1['上り3F偏差値_stdtime1'].fillna(merged_df1['上り3F偏差値_stdtime2'])
merged_df1['上り3F偏差値勝馬差分']=merged_df1['上り3F偏差値']-merged_df1['上り3F偏差値_1着_stdtime1'].fillna(merged_df1['上り3F偏差値_1着_stdtime2'])

# 上り3F順位：基準タイム１差分
merged_df1['上り3F順位上位差分']=merged_df1['上り3F順位']-merged_df1['上り3F順位_stdtime1'].fillna(merged_df1['上り3F順位_stdtime2'])
merged_df1['上り3F順位勝馬差分']=merged_df1['上り3F順位']-merged_df1['上り3F順位_1着_stdtime1'].fillna(merged_df1['上り3F順位_1着_stdtime2'])

# 上り3F順位分布：基準タイム１差分
merged_df1['上り3F順位分布上位差分']=merged_df1['上り3F順位分布']-merged_df1['上り3F順位分布_stdtime1'].fillna(merged_df1['上り3F順位分布_stdtime2'])
merged_df1['上り3F順位分布勝馬差分']=merged_df1['上り3F順位分布']-merged_df1['上り3F順位分布_1着_stdtime1'].fillna(merged_df1['上り3F順位分布_1着_stdtime2'])

# Ave-3F：基準タイム１差分
merged_df1['Ave-3F上位差分']=merged_df1['Ave-3F']-merged_df1['Ave-3F_stdtime1'].fillna(merged_df1['Ave-3F_stdtime2'])
merged_df1['Ave-3F勝馬差分']=merged_df1['Ave-3F']-merged_df1['Ave-3F_1着_stdtime1'].fillna(merged_df1['Ave-3F_1着_stdtime2'])

# Ave-3F偏差値：基準タイム１差分
merged_df1['Ave-3F偏差値上位差分']=merged_df1['Ave-3F偏差値']-merged_df1['Ave-3F偏差値_stdtime1'].fillna(merged_df1['Ave-3F偏差値_stdtime2'])
merged_df1['Ave-3F偏差値勝馬差分']=merged_df1['Ave-3F偏差値']-merged_df1['Ave-3F偏差値_1着_stdtime1'].fillna(merged_df1['Ave-3F偏差値_1着_stdtime2'])

# Ave-3F順位：基準タイム１差分
merged_df1['Ave-3F順位上位差分']=merged_df1['Ave-3F順位']-merged_df1['Ave-3F順位_stdtime1'].fillna(merged_df1['Ave-3F順位_stdtime2'])
merged_df1['Ave-3F順位勝馬差分']=merged_df1['Ave-3F順位']-merged_df1['Ave-3F順位_1着_stdtime1'].fillna(merged_df1['Ave-3F順位_1着_stdtime2'])

# Ave-3F順位分布：基準タイム１差分
merged_df1['Ave-3F順位分布上位差分']=merged_df1['Ave-3F順位分布']-merged_df1['Ave-3F順位分布_stdtime1'].fillna(merged_df1['Ave-3F順位分布_stdtime2'])
merged_df1['Ave-3F順位分布勝馬差分']=merged_df1['Ave-3F順位分布']-merged_df1['Ave-3F順位分布_1着_stdtime1'].fillna(merged_df1['Ave-3F順位分布_1着_stdtime2'])

# -3F差：基準タイム１差分
merged_df1['-3F差上位差分']=merged_df1['-3F差']-merged_df1['-3F差_stdtime1'].fillna(merged_df1['-3F差_stdtime2'])
merged_df1['-3F差勝馬差分']=merged_df1['-3F差']-merged_df1['-3F差_1着_stdtime1'].fillna(merged_df1['-3F差_1着_stdtime2'])

# テン指数：基準タイム１差分
merged_df1['テン指数上位差分']=merged_df1['テン指数']-merged_df1['テン指数_stdtime1'].fillna(merged_df1['テン指数_stdtime2'])
merged_df1['テン指数勝馬差分']=merged_df1['テン指数']-merged_df1['テン指数_1着_stdtime1'].fillna(merged_df1['テン指数_1着_stdtime2'])

# テン指数偏差値：基準タイム１差分
merged_df1['テン指数偏差値上位差分']=merged_df1['テン指数偏差値']-merged_df1['テン指数偏差値_stdtime1'].fillna(merged_df1['テン指数偏差値_stdtime2'])
merged_df1['テン指数偏差値勝馬差分']=merged_df1['テン指数偏差値']-merged_df1['テン指数偏差値_1着_stdtime1'].fillna(merged_df1['テン指数偏差値_1着_stdtime2'])

# テン指数順位：基準タイム１差分
merged_df1['テン指数順位上位差分']=merged_df1['テン指数順位']-merged_df1['テン指数順位_stdtime1'].fillna(merged_df1['テン指数順位_stdtime2'])
merged_df1['テン指数順位勝馬差分']=merged_df1['テン指数順位']-merged_df1['テン指数順位_1着_stdtime1'].fillna(merged_df1['テン指数順位_1着_stdtime2'])

# テン指数順位分布：基準タイム１差分
merged_df1['テン指数順位分布上位差分']=merged_df1['テン指数順位分布']-merged_df1['テン指数順位分布_stdtime1'].fillna(merged_df1['テン指数順位分布_stdtime2'])
merged_df1['テン指数順位分布勝馬差分']=merged_df1['テン指数順位分布']-merged_df1['テン指数順位分布_1着_stdtime1'].fillna(merged_df1['テン指数順位分布_1着_stdtime2'])

# 上り指数：基準タイム１差分
merged_df1['上り指数上位差分']=merged_df1['上り指数']-merged_df1['上り指数_stdtime1'].fillna(merged_df1['上り指数_stdtime2'])
merged_df1['上り指数勝馬差分']=merged_df1['上り指数']-merged_df1['上り指数_1着_stdtime1'].fillna(merged_df1['上り指数_1着_stdtime2'])

# 上り指数偏差値：基準タイム１差分
merged_df1['上り指数偏差値上位差分']=merged_df1['上り指数偏差値']-merged_df1['上り指数偏差値_stdtime1'].fillna(merged_df1['上り指数偏差値_stdtime2'])
merged_df1['上り指数偏差値勝馬差分']=merged_df1['上り指数偏差値']-merged_df1['上り指数偏差値_1着_stdtime1'].fillna(merged_df1['上り指数偏差値_1着_stdtime2'])

# 上り指数順位：基準タイム１差分
merged_df1['上り指数順位上位差分']=merged_df1['上り指数順位']-merged_df1['上り指数順位_stdtime1'].fillna(merged_df1['上り指数順位_stdtime2'])
merged_df1['上り指数順位勝馬差分']=merged_df1['上り指数順位']-merged_df1['上り指数順位_1着_stdtime1'].fillna(merged_df1['上り指数順位_1着_stdtime2'])

# 上り指数順位分布：基準タイム１差分
merged_df1['上り指数順位分布上位差分']=merged_df1['上り指数順位分布']-merged_df1['上り指数順位分布_stdtime1'].fillna(merged_df1['上り指数順位分布_stdtime2'])
merged_df1['上り指数順位分布勝馬差分']=merged_df1['上り指数順位分布']-merged_df1['上り指数順位分布_1着_stdtime1'].fillna(merged_df1['上り指数順位分布_1着_stdtime2'])

# スピード指数：基準タイム１差分
merged_df1['スピード指数上位差分']=merged_df1['スピード指数']-merged_df1['スピード指数_stdtime1'].fillna(merged_df1['スピード指数_stdtime2'])
merged_df1['スピード指数勝馬差分']=merged_df1['スピード指数']-merged_df1['スピード指数_1着_stdtime1'].fillna(merged_df1['スピード指数_1着_stdtime2'])

# スピード指数偏差値：基準タイム１差分
merged_df1['スピード指数偏差値上位差分']=merged_df1['スピード指数偏差値']-merged_df1['スピード指数偏差値_stdtime1'].fillna(merged_df1['スピード指数偏差値_stdtime2'])
merged_df1['スピード指数偏差値勝馬差分']=merged_df1['スピード指数偏差値']-merged_df1['スピード指数偏差値_1着_stdtime1'].fillna(merged_df1['スピード指数偏差値_1着_stdtime2'])

# スピード指数順位：基準タイム１差分
merged_df1['スピード指数順位上位差分']=merged_df1['スピード指数順位']-merged_df1['スピード指数順位_stdtime1'].fillna(merged_df1['スピード指数順位_stdtime2'])
merged_df1['スピード指数順位勝馬差分']=merged_df1['スピード指数順位']-merged_df1['スピード指数順位_1着_stdtime1'].fillna(merged_df1['スピード指数順位_1着_stdtime2'])

# スピード指数順位分布：基準タイム１差分
merged_df1['スピード指数順位分布上位差分']=merged_df1['スピード指数順位分布']-merged_df1['スピード指数順位分布_stdtime1'].fillna(merged_df1['スピード指数順位分布_stdtime2'])
merged_df1['スピード指数順位分布勝馬差分']=merged_df1['スピード指数順位分布']-merged_df1['スピード指数順位分布_1着_stdtime1'].fillna(merged_df1['スピード指数順位分布_1着_stdtime2'])

# 総合指数：基準タイム１差分
merged_df1['総合指数上位差分']=merged_df1['総合指数']-merged_df1['総合指数_stdtime1'].fillna(merged_df1['総合指数_stdtime2'])
merged_df1['総合指数勝馬差分']=merged_df1['総合指数']-merged_df1['総合指数_1着_stdtime1'].fillna(merged_df1['総合指数_1着_stdtime2'])

# 総合指数偏差値：基準タイム１差分
merged_df1['総合指数偏差値上位差分']=merged_df1['総合指数偏差値']-merged_df1['総合指数偏差値_stdtime1'].fillna(merged_df1['総合指数偏差値_stdtime2'])
merged_df1['総合指数偏差値勝馬差分']=merged_df1['総合指数偏差値']-merged_df1['総合指数偏差値_1着_stdtime1'].fillna(merged_df1['総合指数偏差値_1着_stdtime2'])

# 総合指数順位：基準タイム１差分
merged_df1['総合指数順位上位差分']=merged_df1['総合指数順位']-merged_df1['総合指数順位_stdtime1'].fillna(merged_df1['総合指数順位_stdtime2'])
merged_df1['総合指数順位勝馬差分']=merged_df1['総合指数順位']-merged_df1['総合指数順位_1着_stdtime1'].fillna(merged_df1['総合指数順位_1着_stdtime2'])

# 総合指数順位分布：基準タイム１差分
merged_df1['総合指数順位分布上位差分']=merged_df1['総合指数順位分布']-merged_df1['総合指数順位分布_stdtime1'].fillna(merged_df1['総合指数順位分布_stdtime2'])
merged_df1['総合指数順位分布勝馬差分']=merged_df1['総合指数順位分布']-merged_df1['総合指数順位分布_1着_stdtime1'].fillna(merged_df1['総合指数順位分布_1着_stdtime2'])

# PCI：基準タイム１差分
merged_df1['PCI上位差分']=merged_df1['PCI']-merged_df1['PCI_stdtime1'].fillna(merged_df1['PCI_stdtime2'])
merged_df1['PCI勝馬差分']=merged_df1['PCI']-merged_df1['PCI_1着_stdtime1'].fillna(merged_df1['PCI_1着_stdtime2'])

# 初角_4角差：基準タイム１差分
merged_df1['初角_4角差上位差分']=merged_df1['初角_4角差']-merged_df1['初角_4角差_stdtime1'].fillna(merged_df1['初角_4角差_stdtime2'])
merged_df1['初角_4角差勝馬差分']=merged_df1['初角_4角差']-merged_df1['初角_4角差_1着_stdtime1'].fillna(merged_df1['初角_4角差_1着_stdtime2'])

# 4角_入線順位差：基準タイム１差分
merged_df1['4角_入線順位差上位差分']=merged_df1['4角_入線順位差']-merged_df1['4角_入線順位差_stdtime1'].fillna(merged_df1['4角_入線順位差_stdtime2'])
merged_df1['4角_入線順位差勝馬差分']=merged_df1['4角_入線順位差']-merged_df1['4角_入線順位差_1着_stdtime1'].fillna(merged_df1['4角_入線順位差_1着_stdtime2'])

# ---ステップ５：33ラップを判定する
# 33ラップ判定用関数
def assign_label(row):
    lap_value = row['33ラップ']
    if pd.isna(lap_value):
        return np.nan

    candidates = {
        -2: row['33ラップ-2_33lap'],
        -1: row['33ラップ-1_33lap'],
         0: row['33ラップ±0_33lap'],
         1: row['33ラップ+1_33lap'],
         2: row['33ラップ+2_33lap'],
    }

    # 参照側が全部NaNなら判定不能
    if all(pd.isna(v) for v in candidates.values()):
        return np.nan

    # 差分（絶対値）が最小のスケールを選ぶ
    best_scale = min(
        candidates.keys(),
        key=lambda k: abs(lap_value - candidates[k]) if pd.notna(candidates[k]) else np.inf
    )

    # 0スケール
    if best_scale == 0:
        if lap_value < 0:
            return '持0'
        elif lap_value > 0:
            return '瞬0'
        else:
            return '総'

    prefix = '瞬' if lap_value > 0 else '持'
    suffix = f'+{best_scale}' if best_scale > 0 else f'{best_scale}'
    return prefix + suffix

# 33ラップを判定
merged_df1['33ラップ判定'] = merged_df1.apply(assign_label, axis=1)

# レースタイプラベル列の追加
insert_pos = merged_df1.columns.get_loc('33ラップ判定') + 1

# '33ラップ判定' の値 → レースタイプ の対応表
race_type_map = {
    # 瞬発力戦
    '瞬+2': '瞬発力戦',
    '瞬+1': '瞬発力戦',
    '瞬0':  '瞬発力戦',

    # 総合力戦
    '瞬-1': '総合力戦',
    '瞬-2': '総合力戦',
    '総':   '総合力戦',
    '持-1': '総合力戦',
    '持-2': '総合力戦',

    # 持久力戦
    '持+2': '持久力戦',
    '持+1': '持久力戦',
    '持+0': '持久力戦',
}

ref = merged_df1['33ラップ判定']
merged_df1.insert(insert_pos, 'レースタイプ', ref.map(race_type_map))

# ラベル書き換え用のマッピング辞書
label_mapping = {
    '持-2': '0T持-2',
    '持-1': '0T持-1',
    '持0': '0U持0',
    '持+1': '0V持+1',
    '持+2': '0V持+2',
    '瞬-2': '0S瞬-2',
    '瞬-1': '0S瞬-1',
    '瞬0': '0R瞬0',
    '瞬+1': '0Q瞬+1',
    '瞬+2': '0Q瞬+2',
    '総': '02総'
}

# 書き換え
merged_df1['レース印２'] = merged_df1['33ラップ判定'].replace(label_mapping)

# ---ステップ６：レースレベル判定
# レースレベル指数
# レース強度指数の計算
first_load  = merged_df1['通過3F_stdtime1'].fillna(merged_df1['通過3F_stdtime2']) - merged_df1['通過3F']

middle_diff1 = merged_df1['中盤ラップ1_stdtime1'].fillna(merged_df1['中盤ラップ1_stdtime2']) - merged_df1['中盤ラップ1']
middle_diff2 = merged_df1['中盤ラップ2_stdtime1'].fillna(merged_df1['中盤ラップ2_stdtime2']) - merged_df1['中盤ラップ2']

middle_load = pd.concat([middle_diff1, middle_diff2], axis=1).mean(axis=1, skipna=True)

last_load   = merged_df1['レース上り3F_stdtime1'].fillna(merged_df1['レース上り3F_stdtime2']) - merged_df1['レース上り3F']
spurt_load  = merged_df1['最大加速ラップ_stdtime1'].fillna(merged_df1['最大加速ラップ_stdtime2']) - merged_df1['最大加速ラップ']
goal_load   = merged_df1['ゴール前ラップ差_stdtime1'].fillna(merged_df1['ゴール前ラップ差_stdtime2']) - merged_df1['ゴール前ラップ差']

merged_df1['レース強度指数'] = (
    100
    + first_load.fillna(0)
    + middle_load.fillna(0)
    + last_load.fillna(0)
    + spurt_load.fillna(0)
    + goal_load.fillna(0)
    + merged_df1['馬場補正値'].fillna(0)
)

# ベース値（Std2優先 → なければ Std1）
rating_base = merged_df1['レイティング_stdtime2'].fillna(
    merged_df1['レイティング_stdtime1']
)
speed_base = merged_df1['総合指数_stdtime2'].fillna(
    merged_df1['総合指数_stdtime1']
)
strength_base = merged_df1['レース強度指数_stdtime2'].fillna(
    merged_df1['レース強度指数_stdtime1']
)

# 差分（NaN は 0 扱い）
rating_diff = (merged_df1['レイティング平均値'] - rating_base).fillna(0)
speed_diff  = (merged_df1['Top3総合指数'] - speed_base).fillna(0)
strength_diff = (merged_df1['レース強度指数'] - strength_base).fillna(0)

# レースレベル指数算出
merged_df1['レースレベル指数'] = (rating_diff + speed_diff + strength_diff + 100).round(1)

# レース強度指数：基準タイム１差分
merged_df1['レース強度指数上位差分']=merged_df1['レース強度指数']-merged_df1['レース強度指数_stdtime1'].fillna(merged_df1['レース強度指数_stdtime2'])

# 差分
rl_m2 = (merged_df1['レースレベル指数'] - merged_df1['RL-2_racelevel']).abs()
rl_m1 = (merged_df1['レースレベル指数'] - merged_df1['RL-1_racelevel']).abs()
rl_0  = (merged_df1['レースレベル指数'] - merged_df1['RL±0_racelevel']).abs()
rl_p1 = (merged_df1['レースレベル指数'] - merged_df1['RL+1_racelevel']).abs()
rl_p2 = (merged_df1['レースレベル指数'] - merged_df1['RL+2_racelevel']).abs()

# 5本を横に並べて「最小の列名」を取る（行ごと）
diff_df = pd.concat([rl_m2, rl_m1, rl_0, rl_p1, rl_p2], axis=1)
diff_df.columns = [-2, -1, 0, 1, 2]  # そのまま判定値にする

# 「基準が無い行」判定（5本すべてNaN）
no_ref = diff_df.isna().all(axis=1)

# idxminを安定させるため NaN は無限大扱いにして最小を取る
band = diff_df.fillna(np.inf).idxmin(axis=1)

# レースレベル判定 列を追加
insert_pos = merged_df1.columns.get_loc('レースレベル指数') + 1
merged_df1.insert(insert_pos, 'レースレベル判定', band.where(~no_ref, np.nan).astype('Int64'))

# レースレベル評価 列を追加
band_to_grade = {2: 'A', 1: 'B', 0: 'C', -1: 'D', -2: 'E'}
insert_pos = merged_df1.columns.get_loc('レースレベル判定') + 1
merged_df1.insert(insert_pos, 'レースレベル評価', merged_df1['レースレベル判定'].map(band_to_grade))

# レース印３ 列を追加
rank_to_label = {'A': '05A', 'B': '07B', 'C': '01C', 'D': '00D', 'E': '03E'}
merged_df1['レース印３'] = merged_df1['レースレベル評価'].replace(rank_to_label)

# サフィックス付き列を削除
# 削除するサフィックスを列挙
suffixes_to_drop = ('_stdtime1', '_stdtime2', '_paceindex', '_33lap', '_centralscore1', '_centralscore2', '_racelevel')

# サフィックスで判定して残す列だけ抜き出し
merged_df1 = merged_df1.loc[:, [c for c in merged_df1.columns
                              if not any(c.endswith(suf) for suf in suffixes_to_drop)]].copy()

# ---ステップ７：指数系データのcsvファイル保存
# 各指数をインポート用ファイルに加工して保存
# テン指数
imp_df1 = merged_df1[['target_horseid','テン指数']]
# 上り指数
imp_df2 = merged_df1[['target_horseid','上り指数']]
# スピード指数
imp_df3 = merged_df1[['target_horseid','スピード指数']]
# 総合指数
imp_df4 = merged_df1[['target_horseid','総合指数']]
# 33ラップ判定
imp_df5 = merged_df1[['target_raceid','レース印２']].drop_duplicates('target_raceid')
# レースレベル判定
imp_df6 = merged_df1[['target_raceid','レース印３']].drop_duplicates('target_raceid')

output_df1 = pd.read_csv(output_filepath1, encoding='cp932')
output_df2 = pd.read_csv(output_filepath2, encoding='cp932')
output_df3 = pd.read_csv(output_filepath3, encoding='cp932')
output_df4 = pd.read_csv(output_filepath4, encoding='cp932')
output_df5 = pd.read_csv(output_filepath5, encoding='cp932')
output_df6 = pd.read_csv(output_filepath6, encoding='cp932')

# 結合
output_df1 = pd.concat([output_df1, imp_df1], ignore_index=True)
output_df2 = pd.concat([output_df2, imp_df2], ignore_index=True)
output_df3 = pd.concat([output_df3, imp_df3], ignore_index=True)
output_df4 = pd.concat([output_df4, imp_df4], ignore_index=True)
output_df5 = pd.concat([output_df5, imp_df5], ignore_index=True)
output_df6 = pd.concat([output_df6, imp_df6], ignore_index=True)

# 重複削除（最新の行を残す）
output_df1 = output_df1.drop_duplicates(subset=['target_horseid'], keep='last')
output_df2 = output_df2.drop_duplicates(subset=['target_horseid'], keep='last')
output_df3 = output_df3.drop_duplicates(subset=['target_horseid'], keep='last')
output_df4 = output_df4.drop_duplicates(subset=['target_horseid'], keep='last')
output_df5 = output_df5.drop_duplicates(subset=['target_raceid'], keep='last')
output_df6 = output_df6.drop_duplicates(subset=['target_raceid'], keep='last')

# 保存
output_df1.to_csv(output_filepath1, index=False, encoding='cp932')
output_df2.to_csv(output_filepath2, index=False, encoding='cp932')
output_df3.to_csv(output_filepath3, index=False, encoding='cp932')
output_df4.to_csv(output_filepath4, index=False, encoding='cp932')
output_df5.to_csv(output_filepath5, index=False, encoding='cp932')
output_df6.to_csv(output_filepath6, index=False, encoding='cp932')

# 年度列を追加して保存
# 年度列を追加（target_raceid,target_horseid先頭4桁が年）
output_df1['year'] = pd.to_datetime(
    output_df1['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

output_df2['year'] = pd.to_datetime(
    output_df2['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

output_df3['year'] = pd.to_datetime(
    output_df3['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

output_df4['year'] = pd.to_datetime(
    output_df4['target_horseid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

output_df5['year'] = pd.to_datetime(
    output_df5['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

output_df6['year'] = pd.to_datetime(
    output_df6['target_raceid'].astype(str).str.slice(0, 4),
    format='%Y',
    errors='coerce'
).dt.year

# 年度別にファイル分割
for year, df in output_df1.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir1}First_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in output_df2.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir2}Spurt_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in output_df3.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir3}Speed_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in output_df4.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir4}Total_Score_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in output_df5.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir5}33Lap_Category_Master_{year}.csv',
        index=False, encoding='cp932'
    )

for year, df in output_df6.groupby('year'):
    df.drop(columns='year').to_csv(
        f'{output_dir6}Race_Level_Master_{year}.csv',
        index=False, encoding='cp932'
    )

# ---ステップ８：成績データの結合
# 成績データの列の並び替え
header_df = pd.read_csv(header_filepath, header = None, encoding='cp932')
column_list = header_df[0].tolist()
merged_df1 = merged_df1[column_list]

# マスタ成績ファイルを開いてmerged_df1を縦結合
master_df = pd.read_csv(master_filepath, encoding='cp932')

# 結合
master_df = pd.concat([master_df, merged_df1], ignore_index=True)

# 重複削除（最新の行を残す）
master_df = master_df.drop_duplicates(subset=['target_horseid'], keep='last')

# ---ステップ９：馬の持久力/瞬発力タイプ判定をして保存
# master_dfをコピーして１~３着だけにする
merge_key6 = ['場所', '芝・ダート', '距離', 'トラックコード(JV)','馬場分類']

# 馬名も含めて抽出
horsetype_df = master_df[master_df['入線順位'].between(1, 3)].copy()
horsetype_df = horsetype_df[['target_raceid', '血統登録番号', '馬名', 'PCI3', 'PCI'] + merge_key6].copy()

# 基準タイムファイル１から基準PCI3を取ってマージ
std1_df = base_time_df1[merge_key6 + ['PCI3_stdtime1']].copy()
horsetype_df = pd.merge(horsetype_df, std1_df, on=merge_key6, how='left')

# 不要なデータフレームをクリア
del std1_df

# レース性質（基準比）と、馬のレース内差分を合算
# （※この計算式は前回のままでも動きますが、シンプルにするなら horsetype_df['PCI'] - horsetype_df['PCI3_stdtime1'] でも同じ結果です）
horsetype_df['基準PCI3差分'] = horsetype_df['PCI3'] - horsetype_df['PCI3_stdtime1']
horsetype_df['レースPCI3差分'] = horsetype_df['PCI'] - horsetype_df['PCI3']
horsetype_df['PCI判定スコア'] = horsetype_df['基準PCI3差分'] + horsetype_df['レースPCI3差分']

# 馬ごとに集計：スコアは「中央値」、馬名は「最後のもの（最新）」を取得
# ※ここで horsetype_df 自体が「集計後のデータ」に上書き
horsetype_df = horsetype_df.groupby('血統登録番号', as_index=False).agg({
    'PCI判定スコア': 'median',
    '馬名': 'last'
})

# 'タイプ' 列を追加して判定（マイナス=持、プラス=瞬）
horsetype_df['タイプ'] = ''
horsetype_df.loc[horsetype_df['PCI判定スコア'] < 0, 'タイプ'] = '0'
horsetype_df.loc[horsetype_df['PCI判定スコア'] > 0, 'タイプ'] = '1'

# 今日の日付を yymmdd で作る
today_yymmdd = datetime.now().strftime("%y%m%d")

# ★「0/1が入った馬だけ」出力（中央値0や判定不能は除外）
horsetype_df = horsetype_df[horsetype_df['タイプ'].isin(['0', '1'])].copy()

# 登録日
horsetype_df['登録日'] = today_yymmdd

# 血統登録番号の整形
horsetype_df['血統登録番号'] = horsetype_df['血統登録番号'].astype(str).str.strip()

# 列順を整えて完了
checkhorse_df = horsetype_df[['馬名', 'タイプ', '登録日', '血統登録番号']].copy()

# 馬タイプをcsvファイル保存
checkhorse_df.to_csv(output_filepath7, index=False, encoding='cp932')

# ---ステップ１０：成績データファイルを保存
master_df.to_csv(master_filepath, index=False, encoding='cp932')

# 不要なデータフレームのクリア
del base_time_df1
del base_time_df2
del pace_medians_df
del lap33_df
del racelevel_df
del horsetype_df
del master_df

C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\2477927498.py:89: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df1 = pd.read_csv(merged_filepath, encoding='cp932').copy()
C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\2477927498.py:1092: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  merged_df1.insert(insert_pos, 'レースタイプ', ref.map(race_type_map))
C:\Users\rotte\AppData\Local\Temp\ipykernel_18128\2477927498.py:1126: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

# 過去n走分の特徴量を出力する
## 2017~2024年の特徴量を出力する

In [6]:
import sys
import time
import importlib.util
from pathlib import Path
from tqdm import tqdm
    
script_path = Path(r"G:\マイドライブ\20_HOBBY\20_KEIBA\50_SourceCode\add_past_features.py")
spec = importlib.util.spec_from_file_location("add_past_features", script_path)
mod = importlib.util.module_from_spec(spec)

sys.modules[spec.name] = mod
spec.loader.exec_module(mod)

mod.run(
    input_path=r"G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Processed\Race_Result_Master_2024_tmp.csv",
    surface_col="芝・ダート",
    surface_turf_value="芝",
    surface_dirt_value="ダート",
    course_label_col="コースラベル",
)


[info] input: G:\マイドライブ\20_HOBBY\20_KEIBA\10_Data_Source\10_Export_Data\Result_Data\Processed\Race_Result_Master_2024_tmp.csv


G:\マイドライブ\20_HOBBY\20_KEIBA\50_SourceCode\add_past_features.py:948: DtypeWarning: Columns (148) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(cfg.input_path, encoding="cp932")
past_numeric_chunks: 100%|██████████| 4/4 [20:34<00:00, 308.63s/it]


[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Horse\horse_past_numeric_snapshot.csv
[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Horse\horse_runningstyle_snapshot.csv
[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Horse\horse_blinker_flags_snapshot.csv


horse_last5: 100%|██████████| 44759/44759 [04:21<00:00, 171.26it/s]


[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Horse\horse_last5_perf_snapshot.csv
[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Blinker\Horse\blinker_performance_snapshot.csv


...

[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Blinker\Trainer\blinker_performance_snapshot.csv


blinker_rel_keys:  67%|██████▋   | 2/3 [00:02<00:01,  1.06s/it]

[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Blinker\Jockey\blinker_performance_snapshot.csv


blinker_rel_keys: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]

[info] saved: G:\マイドライブ\20_HOBBY\20_KEIBA\20_Data_Features\Blinker\Trainer_Jockey\blinker_performance_snapshot.csv
[info] done


# ↓アーカイブ